In [1]:
import pandas as pd
import numpy as np
import os 
!pip install plotly
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
from datetime import datetime
import scipy as sp
import time
from random import random
pio.renderers.default = "iframe"
def create_spike_trigger(df,col = 'delta_max',lookback_threshold = 3600):
    print('CREATING SPIKE TRIGGER...')
    df[col + '_spike_trigger'] = 0
    col_loc = df.columns.get_loc(col)
    arr = df.values
    for i in range(lookback_threshold,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            cur_max = max(arr[i - lookback_threshold : i ,col_loc])
        except:
            print('ERROR',i)
            cur_max = 100
        if arr[i,col_loc] > cur_max and sum(arr[i - 100 : i,-1]) == 0:
            arr[i,-1] = 1
    return pd.DataFrame(arr,columns = df.columns)

def create_wick_trigger(df,col = 'wick',lookback_threshold = 3600):
    print('CREATING WICK TRIGGER...')
    df['wick'] = 0
    df.loc[df[df['delta_vector'] < 0].index,'wick'] = df.loc[df[df['delta_vector'] < 0].index,'c'] - df.loc[df[df['delta_vector'] < 0].index,'l']
    df.loc[df[df['delta_vector'] > 0].index,'wick'] = df.loc[df[df['delta_vector'] > 0].index,'h'] - df.loc[df[df['delta_vector'] > 0].index,'c']
    df['wick_trigger'] = 0


    col_loc = df.columns.get_loc(col)
    arr = df.values
    for i in range(lookback_threshold,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            cur_max = max(arr[i - lookback_threshold : i ,col_loc])
        except:
            print('ERROR',i)
            cur_max = 100
        if arr[i,col_loc] > cur_max and sum(arr[i - 100 : i,-1]) == 0:
            arr[i,-1] = 1
    return pd.DataFrame(arr,columns = df.columns)
def consecutive_candles(df,num = 10,col = 'delta_vector'):
    print('CREATING CONSECUTIVE CANDLES TRIGGER...')
    df[col + '_consecutive_trigger'] = 0
    col_loc = df.columns.get_loc(col)
    arr = df.values
    last = -1
    counter = 0
    for i in range(arr.shape[0]):
        
        if i % 200000 == 0:
            print(i)
            
        if arr[i-1,col_loc] < 0 and arr[i,col_loc] < 0:
            counter += 1
        elif arr[i-1,col_loc] > 0 and arr[i,col_loc] > 0:
            counter += 1  
        else:
            counter = 0
            
        if counter == num:
            arr[i,-1] = 1
            counter = 0
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE OF TRIGGER DF:',df[df[col + '_consecutive_trigger'] == 1].shape)
    return df

def get_pearsons_corr(df,lookback = 10):
    print('GETTING PEARSONS CORR')
    df['ind'] = list(range(df.shape[0]))
    df['pearsons_corr'] = 0
    ind_col = df.columns.get_loc('ind')
    o_col = df.columns.get_loc('o')
    
    arr = df.values
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        r, p = sp.stats.pearsonr(arr[(i - looback) : i,ind_col], arr[(i - looback) : i,o_col])
        arr[i,-1] = r
    return pd.DataFrame(arr,columns = df.columns)

def get_pearsons_corr(df,lookback = 10):
    print('GETTING PEARSONS CORR')
    df['ind'] = list(range(df.shape[0]))
    df['pearsons_corr'] = 0
    ind_col = df.columns.get_loc('ind')
    o_col = df.columns.get_loc('o')
    
    arr = df.values
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        r, p = sp.stats.pearsonr(arr[(i - lookback) : i,ind_col], arr[(i - lookback) : i,o_col])
        arr[i,-1] = r
    return pd.DataFrame(arr,columns = df.columns)
def get_pearsons_corr2(df,lookback = 10):
    print('GETTING PEARSONS CORR')
    df['ind'] = list(range(df.shape[0]))
    df['pearsons_lookup'] = 0
    df['pearsons_corr2'] = 0
    ind_col = df.columns.get_loc('ind')
    o_col = df.columns.get_loc('o')
    c_col = df.columns.get_loc('c')
    new_col = df.columns.get_loc('pearsons_lookup')
    
    arr = df.values
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        if i % 2 == 0:
            arr[i,new_col] = arr[i,o_col]
        else:
            arr[i,new_col] = arr[i,c_col]        
            
            
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        r, p = sp.stats.pearsonr(arr[(i - lookback) : i,ind_col], arr[(i - lookback) : i,new_col])
        arr[i,-1] = r
    return pd.DataFrame(arr,columns = df.columns)
def convert_timestamp(df):
    time_col = df.columns.get_loc('time')
    arr = df.values 
    for i in range(arr.shape[0]):
        arr[i,time_col] = datetime.strptime(arr[i,time_col][:-4], '%Y-%m-%dT%H:%M:%S.%f')
    return pd.DataFrame(arr,columns = df.columns)
def get_best_fit(df,lookback = 10):

    def get_slope(y1,y2,total_x = 10):
        """y = mx + b"""
        return (y2 - y1) / total_x
    def get_best_fit_vals(y1,y2,total_x = 10):
        m = get_slope(y1,y2,total_x = total_x)
        lst = []
        for i in range(total_x):
            lst.append((m * i) + y1)
        return lst
    def compare_vals(lst,open_list,close_list,total_x):

        for i in range(total_x):
            if lst[i] <= max(open_list[i],close_list[i]) and lst[i] >= min(open_list[i],close_list[i]):
                pass
            else:
                return 0
        return 1
    print('GETTING BEST FIT INDICATOR')
    df['best_fit'] = 0
    o_col = df.columns.get_loc('o')
    c_col = df.columns.get_loc('c')
    new_col = df.columns.get_loc('best_fit')
    
    arr = df.values
    total_x = lookback 
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)

        y1 = np.mean([arr[i - total_x,o_col],arr[i - total_x,c_col]])
        y2 = np.mean([arr[i ,o_col],arr[i ,c_col]])
        
        temp_lst = get_best_fit_vals(y1,y2,total_x)
        arr[i,new_col] = compare_vals(lst = temp_lst,open_list = list(arr[i - total_x:i,o_col]),close_list = list(arr[i - total_x:i,c_col]),total_x = total_x)
            
    
    return pd.DataFrame(arr,columns = df.columns)   

def print_example(df,ind_list,ind = 30,delta = 240,trendline_lookback = 20):
    i = ind_list[ind]   
    print('row loc:',i,' ind:',ind,' delta:',delta)
    fig = go.Figure(data=go.Candlestick(x=df.iloc[i - delta:i + (delta*2),:]['time'],
                        open=df.iloc[i - delta:i + (delta*2),:]['o'],
                        high=df.iloc[i - delta:i + (delta*2),:]['h'],
                        low=df.iloc[i - delta:i + (delta*2),:]['l'],
                        close=df.iloc[i - delta:i + (delta*2),:]['c']))


    fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                  annotation_text="trigger point" , annotation_position="top left",
                  fillcolor="green", opacity=0.25, line_width=0)
    
    fig.add_shape(type='line',
                    x0=df.iloc[i - trendline_lookback,:]['time'],
                    y0=df.iloc[i - trendline_lookback,:]['o'],
                    x1=df.iloc[i,:]['time'],
                    y1=df.iloc[i ,:]['o'],
                    line=dict(color='Red',),
                    xref='x',
                    yref='y'
    )
    
    
    try:
        plot(fig)
    except:
        from plotly.offline import plot
        plot(fig)
def get_max_min_open_or_close(df):
    o_col = df.columns.get_loc('o')
    c_col = df.columns.get_loc('c')
    df['max_o_c'] = 0
    df['min_o_c'] = 0
    max_col = df.columns.get_loc('max_o_c')
    min_col = df.columns.get_loc('min_o_c')    
    arr = df.values
    for i in range(arr.shape[0]):
        arr[i,max_col] = max(arr[i,o_col],arr[i,c_col])
        arr[i,min_col] = min(arr[i,o_col],arr[i,c_col])
    return pd.DataFrame(arr,columns = df.columns)

def channel_indicator(df,lookback = 20):
    """Take the max of a bunch of consecutive candles and find the slope of the line
    Take the min of a bunch of consecutive candles and find the slope of the line
    
    """
    def get_slope(y1,y2,total_x = 10):
        """y = mx + b"""
        return (y2 - y1) / total_x
    def get_best_fit_vals(y1,y2,total_x = 10):
        m = get_slope(y1,y2,total_x = total_x)
        lst = []
        for i in range(total_x):
            lst.append((m * i) + y1)
        return lst
    def compare_vals(lst,check_list,total_x,check_type = 'max'):

        for i in range(total_x):
            if check_type == 'max':
                if lst[i] >= check_list[i]:
                    pass
                else:
                    return 0
            if check_type == 'min':
                if lst[i] <= check_list[i]:
                    pass
                else:
                    return 0
        return 1
    
    
    print('GETTING CHANNEL INDICATOR')
    df['channel_indicator'] = 0
    max_col = df.columns.get_loc('max_o_c')
    min_col = df.columns.get_loc('min_o_c')
    new_col = df.columns.get_loc('channel_indicator')
    df['spread_indicator'] = 0
    spread_col = df.columns.get_loc('spread_indicator')
    
    arr = df.values
    total_x = lookback 
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)

        y1 = arr[i - total_x,max_col]
        y2 = arr[i ,max_col]
        
        temp_lst1 = get_best_fit_vals(y1,y2,total_x)
        check_1 = compare_vals(lst = temp_lst1,
                                      check_list = list(arr[i - total_x:i,min_col]),
                                      total_x = total_x,
                                      check_type = 'max'
                                     )
        
        
        y1 = arr[i - total_x,min_col]
        y2 = arr[i ,min_col]
        
        temp_lst2 = get_best_fit_vals(y1,y2,total_x)
        check_2 = compare_vals(lst = temp_lst2,
                                      check_list = list(arr[i - total_x:i,max_col]),
                                      total_x = total_x,
                                      check_type = 'min'
                                     )   
        spread = np.array(temp_lst1) - np.array(temp_lst2)     
        arr[i,spread_col] = max(spread)
        
        
        if check_1 == 1 and check_2 == 1:
            arr[i,new_col] = 1
    
    return pd.DataFrame(arr,columns = df.columns)  


def get_support(df,lookup_range = 60,stop_range = 20000,lookup_range2 = 200):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookup'] = 0
    df['support_indicator'] = 0
    lookup_col = df.columns.get_loc('support_lookup')
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            if arr[i,c_col] == min(arr[i - lookup_range : i + lookup_range,c_col]):
                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                for j in range(i + lookup_range2,i + stop_range):
                    if arr[j,l_col] <= val and lookup_check == 0:
                        arr[j,new_col] = 1
                        arr[j,lookup_col] = lookup_ind
                        lookup_check = 1
                        break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

def ema(df,num = 14):
    print('GETTING EMA INDICATOR FOR:',num)
    close_col = df.columns.get_loc('c')
    df['ema_' + str(num)] = df['c']
    arr = df.values
    mult = 2/ (num + 1)
    for i in range(num,arr.shape[0]):
        sma = sum(arr[i - num + 1: i + 1,close_col]) / num
        arr[i,-1] = ((arr[i,close_col] - arr[i - 1,-1]) * mult) + arr[i - 1,-1]
    return pd.DataFrame(arr,columns = df.columns)

file = 'XAU_USD_M1_2019-01-01_2022-01-31.csv'
#file = 'GBP_JPY_M1_2019-01-01_2022-01-31.csv'
#file = 'XAU_USD_S5/2020-08-01_2020-09-01.csv'
#file = 'GBP_JPY_S5/2020-08-01_2020-09-01.csv'

file = 'EUR_USD_M1_2019-01-01_2022-01-31.csv'
file = 'GBP_JPY_M1_2019-01-01_2022-01-31.csv'
file = 'EUR_USD_M1_2019-01-01_2022-01-31.csv'
file = 'GBP_JPY_M1_2019-01-01_2022-01-31.csv'

file = 'EUR_USD_M1_2019-01-01_2022-01-31.csv'

df = pd.read_csv(file)
df = convert_timestamp(df)
df = get_max_min_open_or_close(df)
print(df.shape)
df['o'] = df['o'].astype(float)
df['h'] = df['h'].astype(float)
df['l'] = df['l'].astype(float)
df['c'] = df['c'].astype(float)

df = get_support(df,lookup_range = 1500,stop_range = 20000,lookup_range2 = 200)

df = ema(df,num = 14 * 15)

df = ema(df,num = 50 * 15)
df = ema(df,num = 200 * 15)
#df = ema(df,num = 500 * 15)

df.head()




[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
(1090962, 33)
GETTING SUPPORT INDICATOR
0
200000
400000
600000
800000
1000000
TOTAL FUNCTION TIME: 1.1579868793487549  MINUTES
SHAPE (163, 35)
GETTING EMA INDICATOR FOR: 210
GETTING EMA INDICATOR FOR: 750
GETTING EMA INDICATOR FOR: 3000


,time,volume,o,h,l,c,delta,delta_max,delta_vector,max_vector,...,next_10+1,next_20+1,trigger,max_o_c,min_o_c,support_lookup,support_indicator,ema_210,ema_750,ema_3000
0,2019-01-01 22:00:00,1,1.14649,1.14649,1.14649,1.14649,0,0,0,-0,...,-4e-05,0.0001,0,1.14649,1.14649,0,0,1.14649,1.14649,1.14649
1,2019-01-01 22:01:00,1,1.14634,1.14634,1.14634,1.14634,0,0,0,-0,...,0.00014,0.00012,0,1.14634,1.14634,0,0,1.14634,1.14634,1.14634
2,2019-01-01 22:02:00,2,1.14634,1.14634,1.14634,1.14634,0,0,0,-0,...,-0.00026,-0.00034,0,1.14634,1.14634,0,0,1.14634,1.14634,1.14634
3,2019-01-01 22:03:00,1,1.14658,1.14658,1.14658,1.14658,0,0,0,-0,...,-3e-05,-8e-05,0,1.14658,1.14658,0,0,1.14658,1.14658,1.14658
4,2019-01-01 22:05:00,2,1.14658,1.14658,1.14635,1.14635,0.00023,0.00023,-0.00023,-0.00023,...,-4e-05,2e-05,0,1.14658,1.14635,0,0,1.14635,1.14635,1.14635


In [ ]:
def convert_timestamp(df):
    time_col = df.columns.get_loc('time')
    arr = df.values 
    for i in range(arr.shape[0]):
        arr[i,time_col] = datetime.strptime(arr[i,time_col][:-4], '%Y-%m-%dT%H:%M:%S.%f')
    return pd.DataFrame(arr,columns = df.columns)

df['o'] = df['o'].astype(float)
df['h'] = df['h'].astype(float)
df['l'] = df['l'].astype(float)
df['c'] = df['c'].astype(float)

df = convert_timestamp(df)

In [10]:
from random import random

In [4]:
file

'EUR_USD_M1_2019-01-01_2022-01-31.csv'

# SIM

In [2]:
def simulate_results(
    num_trades = 200,
    acc_val = 10000,
    rr = 6,
    risk_size = .02,
    win_rate = .2,
pr = True):
    start = acc_val
    for i in range(num_trades):
        rand = random()
        if rand > win_rate:
            acc_val = acc_val - (acc_val * risk_size)
        else:
            acc_val = acc_val + (acc_val * (risk_size * rr))
    if pr:
        print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
    return round(acc_val)


def strategy_tester_buy6(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print('RUNNING BUY SIMULATOR')
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            for j in range(i,i + 10000):
                if i not in trade_list:
                    if arr[j,l_col] < val - sl:
                        trade_res.append(-10)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,h_col] >= val + tp:
                        trade_res.append(50)
                        trade_list.append(i)
                        wins.append(i)
                        break
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)
    print()

    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    return np.median(vals)


def get_support6(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005 ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookup6'] = 0
    df['support_indicator6'] = 0
    df['entry_point6'] = 0
    lookup_col = df.columns.get_loc('support_lookup6')
    new_col = df.columns.get_loc('support_indicator6')
    entry_col = df.columns.get_loc('entry_point6')
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        if arr[j,l_col] <= val - pip_threshold2 and lookup_check == 0:
      
                                
                              #  if lookup_check == 0 and arr[j,c_col] == max(arr[j - (60 * 8):j + 1,c_col]) and arr[j,c_col] - min(arr[j - (60 * 8):j + 1,c_col]) >= .3:
                                #if arr[j,l_col] <= val and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold:
                            
                            arr[j,entry_col] = val - pip_threshold2
                            arr[j,new_col] = 1
                            arr[j,lookup_col] = lookup_ind
                            lookup_check = 1
                            break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator6'] == 1].shape)
    return df

cur = 'NON JPY'
#cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .3
    pip_threshold = .5
    pip_threshold2 = .1
    drop_threshold = .1
else:
    pip_threshold = .003
    pip_threshold2 = .0015


In [4]:
final_df = pd.read_csv('SIM_' + file)
final_df.head()

,ending_val,pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,sl,tp,num_trades,risk_size
0,18525.0,0.003,0.0015,2000,1500,10000,0.0005,0.0015,200,0.01
1,14835.0,0.003,0.0015,2000,1500,10000,0.0005,0.0020,200,0.01
2,23194.0,0.003,0.0015,2000,1500,10000,0.0005,0.0025,200,0.01
3,25551.5,0.003,0.0015,2000,1500,10000,0.0005,0.0030,200,0.01
4,26323.0,0.003,0.0015,2000,1500,10000,0.0005,0.0040,200,0.01


In [8]:
final_df = pd.read_csv('SIM_' + file)
print(final_df.shape)
final_df.head()

(8064, 10)


,ending_val,pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,sl,tp,num_trades,risk_size
0,18525.0,0.003,0.0015,2000,1500,10000,0.0005,0.0015,200,0.01
1,14835.0,0.003,0.0015,2000,1500,10000,0.0005,0.0020,200,0.01
2,23194.0,0.003,0.0015,2000,1500,10000,0.0005,0.0025,200,0.01
3,25551.5,0.003,0.0015,2000,1500,10000,0.0005,0.0030,200,0.01
4,26323.0,0.003,0.0015,2000,1500,10000,0.0005,0.0040,200,0.01


In [ ]:
cols = [
    'instrument'
    ,'buy_or_sell'
    ,'trade_strategy'
   ,'ending_val'
    ,'starting_val'
    ,'sl'
    ,'tp'
    ,'RR'
    ,'win_rate'
    ,'pip_threshold'
    ,'pip_threshold2'
    ,'lookup_range'
    ,'lookup_range2'
    ,'stop_range'
    ,'num_trades'
    ,'risk_size' 
    
]

[cur,buy_or_sell,trade_strategy,ending_val,starting_val,sl,tp,round(tp / sl,4),round(win_rate,4),pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,num_trades,risk_size]



In [10]:
final_df = final_df.sort_values(by = 'ending_val',ascending = False)
final_df.head(20)

,ending_val,pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,sl,tp,num_trades,risk_size
6399,1175400.0,0.0075,0.0010,2500,1000,10000,0.00075,0.009,200,0.01
6903,634266.0,0.0075,0.0015,2500,1000,10000,0.00075,0.009,200,0.01
6902,421885.0,0.0075,0.0015,2500,1000,10000,0.00075,0.007,200,0.01
6735,364732.5,0.0075,0.0015,2000,1000,10000,0.00075,0.009,200,0.01
6231,364732.5,0.0075,0.0010,2000,1000,10000,0.00075,0.009,200,0.01
6900,236135.0,0.0075,0.0015,2500,1000,10000,0.00075,0.004,200,0.01
6398,232536.0,0.0075,0.0010,2500,1000,10000,0.00075,0.007,200,0.01
7012,229256.0,0.0075,0.0015,2500,2000,10000,0.00075,0.004,200,0.01
6407,225992.0,0.0075,0.0010,2500,1000,10000,0.00100,0.009,200,0.01
6791,222660.5,0.0075,0.0015,2000,1500,10000,0.00075,0.009,200,0.01


(504, 10)

In [ ]:
c = 0
final_list = []
pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]
lookup_ranges = [500,1000,1500,2000,2500]
lookup_range2s = [500,1000,1500,2000,2500]
stop_ranges = [12500,10000,7500,5000,2500]


pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]

lookup_ranges = [1500,2000,2500]
lookup_range2s = [1000,1500,2000]
stop_ranges = [10000]

sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
tps = [.0015,.002,.0025,.003,.004,.005,.007,.009]
num_trades = 200
risk_size = .01
final_df = pd.DataFrame([0])



cols = [
    
   'ending_val'
    ,'pip_threshold'
    ,'pip_threshold2'
    ,'lookup_range'
    ,'lookup_range2'
    ,'stop_range'
    ,'sl'
    ,'tp'
    ,'num_trades'
    ,'risk_size' 
    
]


final_df = pd.read_csv('SIM_' + file)
for pip_threshold in pip_thresholds:
    for pip_threshold2 in pip_threshold2s:
        for lookup_range in lookup_ranges:
            for lookup_range2 in lookup_range2s:
                for stop_range in stop_ranges:
                    c+=1
                    print()
                    print('LOOP NUM:',c,' pip threshold:',pip_threshold,' pip threshold2:',pip_threshold2,' lookup range:',lookup_range,' lookup range2:',lookup_range2,' stop range:',stop_range)
                    df = get_support6(df,lookup_range = lookup_range,lookup_range2 = lookup_range2,stop_range = stop_range,pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2)
                    temp_lst = []
                    for sl in sls:
                        for tp in tps:
                            ending_val = strategy_tester_buy6(df,col = 'support_indicator6',sl = sl,tp = tp,num_trades = 250,risk_size = .01,entry = 'c',pr = False)
                            temp_lst.append([ending_val,pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,sl,tp,num_trades,risk_size])
                            final_list.append([ending_val,pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,sl,tp,num_trades,risk_size])

                    temp_df = pd.DataFrame(temp_lst,columns = cols)
                    if final_df.shape[0] == 1:
                        final_df = temp_df
                    else:
                        final_df = final_df.append(temp_df)
                    final_df.to_csv('SIM_' + file,index = False)
                            
                            
final_list          




In [3]:
final_df = pd.DataFrame([0])
final_df.shape[0]

1

In [ ]:
c = 0
final_list = []
pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]
lookup_ranges = [500,1000,1500,2000,2500]
lookup_range2s = [500,1000,1500,2000,2500]
stop_ranges = [12500,10000,7500,5000,2500]


pip_threshold2s = [.0005]
pip_thresholds = [.003]

lookup_ranges = [1500]
lookup_range2s = [1000]
stop_ranges = [10000]

sls = [.0005,.00075,.001]
tps = [.0015,.002,.0025]
num_trades = 200
risk_size = .01
final_df = pd.DataFrame([0])
cols = [
    
   'ending_val'
    ,'pip_threshold'
    ,'pip_threshold2'
    ,'lookup_range'
    ,'lookup_range2'
    ,'stop_range'
    ,'sl'
    ,'tp'
    ,'num_trades'
    ,'risk_size' 
    
]
for pip_threshold in pip_thresholds:
    for pip_threshold2 in pip_threshold2s:
        for lookup_range in lookup_ranges:
            for lookup_range2 in lookup_range2s:
                for stop_range in stop_ranges:
                    c+=1
                    print()
                    print('LOOP NUM:',c,' pip threshold:',pip_threshold,' pip threshold2:',pip_threshold2,' lookup range:',lookup_range,' lookup range2:',lookup_range2,' stop range:',stop_range)
                    df = get_support6(df,lookup_range = lookup_range,lookup_range2 = lookup_range2,stop_range = stop_range,pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2)
                    temp_lst = []
                    for sl in sls:
                        for tp in tps:
                            ending_val = strategy_tester_buy6(df,col = 'support_indicator6',sl = sl,tp = tp,num_trades = 250,risk_size = .01,entry = 'c',pr = False)
                            temp_lst.append([ending_val,pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,sl,tp,num_trades,risk_size])
                            final_list.append([ending_val,pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,sl,tp,num_trades,risk_size])
                            
                    temp_df = pd.DataFrame(temp_lst,columns = cols)
                    if final_df.shape[0] == 1:
                        final_df = temp_df
                    else:
                        final_df = final_df.append(temp_df)
                    final_df.to_csv('SIM_' + file,index = False)
                            
                            
final_list          


In [11]:
ending_val = strategy_tester_buy6(df,col = 'support_indicator6',sl = sl,tp = tp,num_trades = 250,risk_size = .01,entry = 'c',pr = False)

RUNNING BUY SIMULATOR
SHAPE: (178, 41)

MEDIAN SIM: 13766.5 AVG SIM: 14428.533333333333 WIN RATE 0.28651685393258425  RR: 3.0 NUM TRADES: 250  RISK SIZE: 0.01


In [1]:
1500 /60

25.0

In [211]:
def viz_support(df,start,end):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240):
        i = ind_list[ind]   
        x = df['support_lookup'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point" , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )

        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_210'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_750'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_3000'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator'] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 1000)
        i += 1
        
        
viz_support(df = df,start = 50,end = 70)



SELL INTO SUPPORT!!!!!
SHAPE: (163, 38)
50
SUPPORT ROW: 277891  SUPPORT NEXT TOUCH: 279936  SUPPORT CLOSE VAL: 1.10737
51
SUPPORT ROW: 283928  SUPPORT NEXT TOUCH: 285491  SUPPORT CLOSE VAL: 1.11306
52
SUPPORT ROW: 281685  SUPPORT NEXT TOUCH: 286571  SUPPORT CLOSE VAL: 1.10806
53
SUPPORT ROW: 286839  SUPPORT NEXT TOUCH: 288376  SUPPORT CLOSE VAL: 1.10638
54
SUPPORT ROW: 290550  SUPPORT NEXT TOUCH: 290883  SUPPORT CLOSE VAL: 1.1017299999999999
55
SUPPORT ROW: 300167  SUPPORT NEXT TOUCH: 301740  SUPPORT CLOSE VAL: 1.10532
56
SUPPORT ROW: 303883  SUPPORT NEXT TOUCH: 305943  SUPPORT CLOSE VAL: 1.10043
57
SUPPORT ROW: 306349  SUPPORT NEXT TOUCH: 308548  SUPPORT CLOSE VAL: 1.09928
58
SUPPORT ROW: 295239  SUPPORT NEXT TOUCH: 308549  SUPPORT CLOSE VAL: 1.0989799999999998
59
SUPPORT ROW: 336835  SUPPORT NEXT TOUCH: 340613  SUPPORT CLOSE VAL: 1.1125399999999999
60
SUPPORT ROW: 346116  SUPPORT NEXT TOUCH: 350147  SUPPORT CLOSE VAL: 1.1105399999999999
61
SUPPORT ROW: 343242  SUPPORT NEXT TOUCH: 350

In [336]:
def simulate_results(
    num_trades = 200,
    acc_val = 10000,
    rr = 6,
    risk_size = .02,
    win_rate = .2):
    start = acc_val
    for i in range(num_trades):
        rand = random()
        if rand > win_rate:
            acc_val = acc_val - (acc_val * risk_size)
        else:
            acc_val = acc_val + (acc_val * (risk_size * rr))
    print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
    return round(acc_val)


def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,entry = 'c'):
    print('RUNNING BUY SIMULATOR')
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            for j in range(i,i + 10000):
                if i not in trade_list:
                    if arr[j,l_col] < val - sl:
                        trade_res.append(-10)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,h_col] >= val + tp:
                        trade_res.append(50)
                        trade_list.append(i)
                        wins.append(i)
                        break
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
    print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades)
    vals = []


    acc_val = 10000

    risk_size = .02

    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate)
        vals.append(final_val)
    print()
    print('AVG SIM:',sum(vals) / len(vals))    
    print('MEDIAN SIM:',np.median(vals))
    
    return trade_res,trade_list
def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300):
    print('RUNNING SELL SIMULATOR')
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        if arr[i,col_num] == 1:
            val = arr[i,c_col]
            for j in range(i,i + 10000):
                if i not in trade_list:
                    if arr[j,l_col] > val + sl:
                        trade_res.append(-10)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,h_col] <= val - tp:
                        trade_res.append(50)
                        trade_list.append(i)
                        wins.append(i)
                        break
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
    #num_trades = len(trade_list)
    print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades)
    vals = []


    acc_val = 10000

    risk_size = .02

    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate)
        vals.append(final_val)
    print()
    print('AVG SIM:',sum(vals) / len(vals))  
    print('MEDIAN SIM:',np.median(vals))
    
    return trade_res,trade_list



#trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator',sl = .001,tp = .003)    
#trade_res,trade_list = strategy_tester_sell(df,col = 'support_indicator',sl = .001,tp = .003) 



# IDEA 2
## Support is Low of last 24 hours that moves up at least 50 PIPs and back down within a window of 12 - 36 hours

In [238]:


def get_support2(df,lookup_range = 1500,lookup_range2 = 700,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .002 ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    df['support_lookup2'] = 0
    df['support_indicator2'] = 0
    lookup_col = df.columns.get_loc('support_lookup2')
    new_col = df.columns.get_loc('support_indicator2')
    arr = df.values
    
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                for j in range(i + lookup_range2,i + stop_range):
                    if min(arr[i + lookup_range2:j - 120,l_col]) >= val:
                        if min(arr[j - 60:j,l_col]) <= val - pip_threshold and arr[j,h_col] >= val + pip_threshold2 and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold:

                            arr[j,new_col] = 1
                            arr[j,lookup_col] = lookup_ind
                            lookup_check = 1
                            break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator2'] == 1].shape)
    return df
cur = 'NON JPY'
cur = 'JPY'
cur = 'NON JPY'
print(cur)

if cur == 'JPY':
    pip_threshold = .5
else:
    pip_threshold = .003
    pip_threshold2 = .001
    
df = get_support2(df,lookup_range = 1500,lookup_range2 = 700,stop_range = 5000,pip_threshold = pip_threshold ,pip_threshold2 = pip_threshold2 )

 


NON JPY
GETTING SUPPORT INDICATOR
0
200000
400000
600000
800000
1000000
TOTAL FUNCTION TIME: 0.8628884315490722  MINUTES
SHAPE (0, 40)


In [236]:
trade_res,trade_list = strategy_tester(df,col = 'support_indicator2',sl = .001,tp = .001) 



SHAPE: (15, 40)
0
200000
400000
600000
800000
1000000
WIN RATE 0.2  RR: 1.0 NUM TRADES: 15
STARTING VAL: 10000  ENDING VAL: 8667
STARTING VAL: 10000  ENDING VAL: 7687
STARTING VAL: 10000  ENDING VAL: 8327
STARTING VAL: 10000  ENDING VAL: 8667
STARTING VAL: 10000  ENDING VAL: 8001
STARTING VAL: 10000  ENDING VAL: 9021
STARTING VAL: 10000  ENDING VAL: 8667
STARTING VAL: 10000  ENDING VAL: 8327
STARTING VAL: 10000  ENDING VAL: 8327
STARTING VAL: 10000  ENDING VAL: 7687
STARTING VAL: 10000  ENDING VAL: 9021
STARTING VAL: 10000  ENDING VAL: 9021
STARTING VAL: 10000  ENDING VAL: 8667
STARTING VAL: 10000  ENDING VAL: 8667
STARTING VAL: 10000  ENDING VAL: 8001
STARTING VAL: 10000  ENDING VAL: 9389
STARTING VAL: 10000  ENDING VAL: 9021
STARTING VAL: 10000  ENDING VAL: 9773
STARTING VAL: 10000  ENDING VAL: 9021
STARTING VAL: 10000  ENDING VAL: 7687
STARTING VAL: 10000  ENDING VAL: 8001
STARTING VAL: 10000  ENDING VAL: 8327
STARTING VAL: 10000  ENDING VAL: 8327
STARTING VAL: 10000  ENDING VAL: 80

In [237]:
def viz_support2(df,start,end):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240):
        i = ind_list[ind]   
        x = df['support_lookup2'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point" , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )

        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_210'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_750'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_3000'],line=dict(color='Black',)))
        #fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_7500'],line=dict(color='Orange',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator2'] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 1000)
        i += 1
        
        
viz_support2(df = df,start = 0,end = 15)


SELL INTO SUPPORT!!!!!
SHAPE: (15, 40)
0
SUPPORT ROW: 20272  SUPPORT NEXT TOUCH: 23016  SUPPORT CLOSE VAL: 1.13386
1
SUPPORT ROW: 226131  SUPPORT NEXT TOUCH: 228486  SUPPORT CLOSE VAL: 1.0963399999999999
2
SUPPORT ROW: 236322  SUPPORT NEXT TOUCH: 237542  SUPPORT CLOSE VAL: 1.0986200000000002
3
SUPPORT ROW: 232939  SUPPORT NEXT TOUCH: 237544  SUPPORT CLOSE VAL: 1.10158
4
SUPPORT ROW: 403256  SUPPORT NEXT TOUCH: 404145  SUPPORT CLOSE VAL: 1.12519
5
SUPPORT ROW: 401682  SUPPORT NEXT TOUCH: 404150  SUPPORT CLOSE VAL: 1.1275
6
SUPPORT ROW: 408568  SUPPORT NEXT TOUCH: 410475  SUPPORT CLOSE VAL: 1.09582
7
SUPPORT ROW: 410130  SUPPORT NEXT TOUCH: 411307  SUPPORT CLOSE VAL: 1.0804200000000002
8
SUPPORT ROW: 635439  SUPPORT NEXT TOUCH: 640145  SUPPORT CLOSE VAL: 1.1650399999999999
9
SUPPORT ROW: 728406  SUPPORT NEXT TOUCH: 732443  SUPPORT CLOSE VAL: 1.20119
10
SUPPORT ROW: 825989  SUPPORT NEXT TOUCH: 827712  SUPPORT CLOSE VAL: 1.21254
11
SUPPORT ROW: 877336  SUPPORT NEXT TOUCH: 879092  SUPPORT C

In [42]:
2200/60

36.666666666666664

# IDEA 3 
## Same as idea 2 but needs to drop a certain amount below support to trigger a buy

In [43]:


def get_support3(df,lookup_range = 1500,lookup_range2 = 700,stop_range = 2200,pip_threshold = .005,drop_threshold = .0015,drop_timeframe = 100 ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookup3'] = 0
    df['support_indicator3'] = 0
    lookup_col = df.columns.get_loc('support_lookup3')
    new_col = df.columns.get_loc('support_indicator3')
    arr = df.values
    
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                for j in range(i + lookup_range2,i + stop_range):
                    if arr[j,l_col] <= val and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold:
                        if min(arr[j:j + drop_timeframe,l_col]) <= val - drop_threshold:
                        
                            arr[j,new_col] = 1
                            arr[j,lookup_col] = lookup_ind
                            lookup_check = 1
                            break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator3'] == 1].shape)
    return df
cur = 'NON JPY'
cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .5
    drop_threshold = .1
else:
    pip_threshold = .005
    drop_threshold = .001
    
df = get_support3(df,lookup_range = 1500,lookup_range2 = 700,stop_range = 4000,pip_threshold = pip_threshold,drop_threshold = drop_threshold,drop_timeframe = 60  )
df.head()


GETTING SUPPORT INDICATOR
0
200000
400000
600000
800000
1000000
TOTAL FUNCTION TIME: 0.8926031986872355  MINUTES
SHAPE (161, 43)


,time,volume,o,h,l,c,delta,delta_max,delta_vector,max_vector,...,support_lookup,support_indicator,ema_210,ema_750,ema_3000,ema_7500,support_lookup2,support_indicator2,support_lookup3,support_indicator3
0,2019-01-01 22:00:00,1,139.822,139.822,139.822,139.822,0,0,0,-0,...,0,0,139.822,139.822,139.822,139.822,0,0,0,0
1,2019-01-01 22:01:00,5,139.778,139.824,139.778,139.824,0.046,0.046,0.046,0.046,...,0,0,139.824,139.824,139.824,139.824,0,0,0,0
2,2019-01-01 22:02:00,3,139.824,139.824,139.806,139.806,0.018,0.018,-0.018,-0.018,...,0,0,139.806,139.806,139.806,139.806,0,0,0,0
3,2019-01-01 22:03:00,1,139.802,139.802,139.802,139.802,0,0,0,-0,...,0,0,139.802,139.802,139.802,139.802,0,0,0,0
4,2019-01-01 22:04:00,5,139.806,139.806,139.76,139.76,0.046,0.046,-0.046,-0.046,...,0,0,139.76,139.76,139.76,139.76,0,0,0,0


In [44]:
def viz_support3(df,start,end):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240):
        i = ind_list[ind]   
        x = df['support_lookup3'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point" , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )

        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_210'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_750'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_3000'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator3'] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 1000)
        i += 1
        
        
viz_support3(df = df,start = 40,end = 70)

SELL INTO SUPPORT!!!!!
SHAPE: (161, 43)
40
SUPPORT ROW: 236440  SUPPORT NEXT TOUCH: 238701  SUPPORT CLOSE VAL: 128.768


/opt/anaconda3/lib/python3.8/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




41
SUPPORT ROW: 240641  SUPPORT NEXT TOUCH: 242200  SUPPORT CLOSE VAL: 129.012
42
SUPPORT ROW: 265045  SUPPORT NEXT TOUCH: 266919  SUPPORT CLOSE VAL: 134.29
43
SUPPORT ROW: 268020  SUPPORT NEXT TOUCH: 270641  SUPPORT CLOSE VAL: 133.364
44
SUPPORT ROW: 270868  SUPPORT NEXT TOUCH: 271919  SUPPORT CLOSE VAL: 132.865
45
SUPPORT ROW: 273287  SUPPORT NEXT TOUCH: 276492  SUPPORT CLOSE VAL: 132.341
46
SUPPORT ROW: 276624  SUPPORT NEXT TOUCH: 277604  SUPPORT CLOSE VAL: 131.80200000000002
47
SUPPORT ROW: 277655  SUPPORT NEXT TOUCH: 278897  SUPPORT CLOSE VAL: 131.605
48
SUPPORT ROW: 278908  SUPPORT NEXT TOUCH: 280587  SUPPORT CLOSE VAL: 131.502
49
SUPPORT ROW: 280834  SUPPORT NEXT TOUCH: 283314  SUPPORT CLOSE VAL: 131.304
50
SUPPORT ROW: 298393  SUPPORT NEXT TOUCH: 300562  SUPPORT CLOSE VAL: 139.092
51
SUPPORT ROW: 310702  SUPPORT NEXT TOUCH: 313746  SUPPORT CLOSE VAL: 139.851
52
SUPPORT ROW: 313854  SUPPORT NEXT TOUCH: 316678  SUPPORT CLOSE VAL: 139.514
53
SUPPORT ROW: 326908  SUPPORT NEXT TOUCH

# Idea 4 sell support?

In [62]:


def get_support4(df,lookup_range = 1500,lookup_range2 = 700,stop_range = 2200,pip_threshold = .005,drop_threshold = .0015,drop_timeframe = 100 ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookup4'] = 0
    df['support_indicator4'] = 0
    lookup_col = df.columns.get_loc('support_lookup4')
    new_col = df.columns.get_loc('support_indicator4')
    arr = df.values
    
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                for j in range(i + lookup_range2,i + stop_range):
                    if arr[j,l_col] <= val and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold:

                        
                        arr[j,new_col] = 1
                        arr[j,lookup_col] = lookup_ind
                        lookup_check = 1
                        break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator4'] == 1].shape)
    return df

cur = 'NON JPY'
cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .3
    drop_threshold = .1
else:
    pip_threshold = .005
    drop_threshold = .001
    
df = get_support4(df,lookup_range = 1500,lookup_range2 = 700,stop_range = 4000,pip_threshold = pip_threshold,drop_threshold = drop_threshold,drop_timeframe = 60  )
df.head()


GETTING SUPPORT INDICATOR
0
200000
400000
600000
800000
1000000
TOTAL FUNCTION TIME: 0.8576314647992452  MINUTES
SHAPE (200, 47)


,time,volume,o,h,l,c,delta,delta_max,delta_vector,max_vector,...,ema_3000,ema_7500,support_lookup2,support_indicator2,support_lookup3,support_indicator3,support_lookup4,support_indicator4,support_lookup5,support_indicator5
0,2019-01-01 22:00:00,1,139.822,139.822,139.822,139.822,0,0,0,-0,...,139.822,139.822,0,0,0,0,0,0,0,0
1,2019-01-01 22:01:00,5,139.778,139.824,139.778,139.824,0.046,0.046,0.046,0.046,...,139.824,139.824,0,0,0,0,0,0,0,0
2,2019-01-01 22:02:00,3,139.824,139.824,139.806,139.806,0.018,0.018,-0.018,-0.018,...,139.806,139.806,0,0,0,0,0,0,0,0
3,2019-01-01 22:03:00,1,139.802,139.802,139.802,139.802,0,0,0,-0,...,139.802,139.802,0,0,0,0,0,0,0,0
4,2019-01-01 22:04:00,5,139.806,139.806,139.76,139.76,0.046,0.046,-0.046,-0.046,...,139.76,139.76,0,0,0,0,0,0,0,0


In [79]:
def viz_support4(df,start,end):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240):
        i = ind_list[ind]   
        x = df['support_lookup4'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point" , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )

        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_210'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_750'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_3000'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator4'] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 1000)
        i += 1
        
        
viz_support4(df = df,start = 100,end = 130)



SELL INTO SUPPORT!!!!!
SHAPE: (200, 47)
100
SUPPORT ROW: 547700  SUPPORT NEXT TOUCH: 548763  SUPPORT CLOSE VAL: 132.059
101
SUPPORT ROW: 561349  SUPPORT NEXT TOUCH: 564233  SUPPORT CLOSE VAL: 134.29399999999998
102
SUPPORT ROW: 572794  SUPPORT NEXT TOUCH: 575559  SUPPORT CLOSE VAL: 135.28799999999998
103
SUPPORT ROW: 596870  SUPPORT NEXT TOUCH: 598347  SUPPORT CLOSE VAL: 139.352
104
SUPPORT ROW: 599472  SUPPORT NEXT TOUCH: 602566  SUPPORT CLOSE VAL: 138.624
105
SUPPORT ROW: 602570  SUPPORT NEXT TOUCH: 604321  SUPPORT CLOSE VAL: 138.588
106
SUPPORT ROW: 615809  SUPPORT NEXT TOUCH: 617183  SUPPORT CLOSE VAL: 141.02
107
SUPPORT ROW: 617217  SUPPORT NEXT TOUCH: 618513  SUPPORT CLOSE VAL: 140.505
108
SUPPORT ROW: 618584  SUPPORT NEXT TOUCH: 619652  SUPPORT CLOSE VAL: 140.13
109
SUPPORT ROW: 619931  SUPPORT NEXT TOUCH: 620789  SUPPORT CLOSE VAL: 139.629
110
SUPPORT ROW: 622358  SUPPORT NEXT TOUCH: 624250  SUPPORT CLOSE VAL: 136.75799999999998
111
SUPPORT ROW: 624521  SUPPORT NEXT TOUCH: 6253

In [60]:
df.head()

,time,volume,o,h,l,c,delta,delta_max,delta_vector,max_vector,...,ema_3000,ema_7500,support_lookup2,support_indicator2,support_lookup3,support_indicator3,support_lookup4,support_indicator4,support_lookup5,support_indicator5
0,2019-01-01 22:00:00,1,139.822,139.822,139.822,139.822,0,0,0,-0,...,139.822,139.822,0,0,0,0,0,0,0,0
1,2019-01-01 22:01:00,5,139.778,139.824,139.778,139.824,0.046,0.046,0.046,0.046,...,139.824,139.824,0,0,0,0,0,0,0,0
2,2019-01-01 22:02:00,3,139.824,139.824,139.806,139.806,0.018,0.018,-0.018,-0.018,...,139.806,139.806,0,0,0,0,0,0,0,0
3,2019-01-01 22:03:00,1,139.802,139.802,139.802,139.802,0,0,0,-0,...,139.802,139.802,0,0,0,0,0,0,0,0
4,2019-01-01 22:04:00,5,139.806,139.806,139.76,139.76,0.046,0.046,-0.046,-0.046,...,139.76,139.76,0,0,0,0,0,0,0,0


# Idea 5 double bottom within 24 hours

In [241]:


def get_support5(df,lookup_range = 1500,stop_range = 2200,pip_threshold = .005 ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookup5'] = 0
    df['support_indicator5'] = 0
    lookup_col = df.columns.get_loc('support_lookup5')
    new_col = df.columns.get_loc('support_indicator5')
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)

    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + 30,c_col])
            
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                for j in range(i + 30,i + stop_range):
                    #print('VAL',(max(arr[i : j,c_col]) - min_))
                    if arr[j,l_col] <= val and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold:
                    #if arr[j,l_col] <= val and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold:

                        arr[j,new_col] = 1
                        arr[j,lookup_col] = lookup_ind
                        lookup_check = 1
                        break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator5'] == 1].shape)
    return df

cur = 'NON JPY'
#cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .3
    pip_threshold = .3
    drop_threshold = .1
else:
    pip_threshold = .005
    drop_threshold = .0005
    
df = get_support5(df,lookup_range = 1500,stop_range = 1500,pip_threshold = pip_threshold)
df.head()


GETTING SUPPORT INDICATOR
STOP RANGE: 1500
0
200000
400000
600000
800000
1000000
TOTAL FUNCTION TIME: 0.8018077691396077  MINUTES
SHAPE (75, 42)


,time,volume,o,h,l,c,delta,delta_max,delta_vector,max_vector,...,min_o_c,support_lookup,support_indicator,ema_210,ema_750,ema_3000,support_lookup2,support_indicator2,support_lookup5,support_indicator5
0,2019-01-01 22:00:00,1,1.14649,1.14649,1.14649,1.14649,0,0,0,-0,...,1.14649,0,0,1.14649,1.14649,1.14649,0,0,0,0
1,2019-01-01 22:01:00,1,1.14634,1.14634,1.14634,1.14634,0,0,0,-0,...,1.14634,0,0,1.14634,1.14634,1.14634,0,0,0,0
2,2019-01-01 22:02:00,2,1.14634,1.14634,1.14634,1.14634,0,0,0,-0,...,1.14634,0,0,1.14634,1.14634,1.14634,0,0,0,0
3,2019-01-01 22:03:00,1,1.14658,1.14658,1.14658,1.14658,0,0,0,-0,...,1.14658,0,0,1.14658,1.14658,1.14658,0,0,0,0
4,2019-01-01 22:05:00,2,1.14658,1.14658,1.14635,1.14635,0.00023,0.00023,-0.00023,-0.00023,...,1.14635,0,0,1.14635,1.14635,1.14635,0,0,0,0


In [243]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator5',sl = .001,tp = .003) 

RUNNING BUY SIMULATOR
SHAPE: (75, 42)
0
200000
400000
600000
800000
1000000
WIN RATE 0.22666666666666666  RR: 3.0 NUM TRADES: 75
STARTING VAL: 10000  ENDING VAL: 8343
STARTING VAL: 10000  ENDING VAL: 7131
STARTING VAL: 10000  ENDING VAL: 5210
STARTING VAL: 10000  ENDING VAL: 8343
STARTING VAL: 10000  ENDING VAL: 9760
STARTING VAL: 10000  ENDING VAL: 9024
STARTING VAL: 10000  ENDING VAL: 5210
STARTING VAL: 10000  ENDING VAL: 15630
STARTING VAL: 10000  ENDING VAL: 9024
STARTING VAL: 10000  ENDING VAL: 12351
STARTING VAL: 10000  ENDING VAL: 12351
STARTING VAL: 10000  ENDING VAL: 7713
STARTING VAL: 10000  ENDING VAL: 7713
STARTING VAL: 10000  ENDING VAL: 4817
STARTING VAL: 10000  ENDING VAL: 6095
STARTING VAL: 10000  ENDING VAL: 7131
STARTING VAL: 10000  ENDING VAL: 8343
STARTING VAL: 10000  ENDING VAL: 9024
STARTING VAL: 10000  ENDING VAL: 6593
STARTING VAL: 10000  ENDING VAL: 9024
STARTING VAL: 10000  ENDING VAL: 9024
STARTING VAL: 10000  ENDING VAL: 8343
STARTING VAL: 10000  ENDING VAL:

In [77]:
def viz_support5(df,start,end):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240):
        i = ind_list[ind]   
        x = df['support_lookup5'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point" , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )

        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_210'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_750'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_3000'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator5'] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 1000)
        i += 1
        
        
viz_support5(df = df,start = 10,end = 40)



SELL INTO SUPPORT!!!!!
SHAPE: (64, 47)
10
SUPPORT ROW: 27707  SUPPORT NEXT TOUCH: 28434  SUPPORT CLOSE VAL: 143.414


/opt/anaconda3/lib/python3.8/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




11
SUPPORT ROW: 28443  SUPPORT NEXT TOUCH: 28481  SUPPORT CLOSE VAL: 142.873
12
SUPPORT ROW: 28488  SUPPORT NEXT TOUCH: 29930  SUPPORT CLOSE VAL: 142.756
13
SUPPORT ROW: 29931  SUPPORT NEXT TOUCH: 30158  SUPPORT CLOSE VAL: 142.744
14
SUPPORT ROW: 30471  SUPPORT NEXT TOUCH: 30597  SUPPORT CLOSE VAL: 142.705
15
SUPPORT ROW: 30701  SUPPORT NEXT TOUCH: 30929  SUPPORT CLOSE VAL: 142.327
16
SUPPORT ROW: 30598  SUPPORT NEXT TOUCH: 31197  SUPPORT CLOSE VAL: 142.704
17
SUPPORT ROW: 30976  SUPPORT NEXT TOUCH: 32120  SUPPORT CLOSE VAL: 142.248
18
SUPPORT ROW: 34725  SUPPORT NEXT TOUCH: 34938  SUPPORT CLOSE VAL: 143.14600000000002
19
SUPPORT ROW: 35281  SUPPORT NEXT TOUCH: 35942  SUPPORT CLOSE VAL: 142.186
20
SUPPORT ROW: 35339  SUPPORT NEXT TOUCH: 35945  SUPPORT CLOSE VAL: 142.061
21
SUPPORT ROW: 45240  SUPPORT NEXT TOUCH: 45794  SUPPORT CLOSE VAL: 141.315
22
SUPPORT ROW: 45882  SUPPORT NEXT TOUCH: 46299  SUPPORT CLOSE VAL: 141.108
23
SUPPORT ROW: 45930  SUPPORT NEXT TOUCH: 46300  SUPPORT CLOSE V

In [82]:
2400 / 60

40.0

# Idea 6 within ~20 pips 2 - 5 days later

In [354]:
def simulate_results(
    num_trades = 200,
    acc_val = 10000,
    rr = 6,
    risk_size = .02,
    win_rate = .2,
pr = True):
    start = acc_val
    for i in range(num_trades):
        rand = random()
        if rand > win_rate:
            acc_val = acc_val - (acc_val * risk_size)
        else:
            acc_val = acc_val + (acc_val * (risk_size * rr))
    if pr:
        print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
    return round(acc_val)


def strategy_tester_buy6(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True):
    print('RUNNING BUY SIMULATOR')
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            for j in range(i,i + 10000):
                if i not in trade_list:
                    if arr[j,l_col] < val - sl:
                        trade_res.append(-10)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,h_col] >= val + tp:
                        trade_res.append(50)
                        trade_list.append(i)
                        wins.append(i)
                        break
    win_rate = len(wins) / len(trade_list)
    rr = tp / sl
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)
    print()

    print('MEDIAN SIM:',np.median(vals),'AVG SIM:',sum(vals) / len(vals),'WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    return np.median(vals)


def get_support6(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005 ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookup6'] = 0
    df['support_indicator6'] = 0
    df['entry_point6'] = 0
    lookup_col = df.columns.get_loc('support_lookup6')
    new_col = df.columns.get_loc('support_indicator6')
    entry_col = df.columns.get_loc('entry_point6')
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        if arr[j,l_col] <= val - pip_threshold2 and lookup_check == 0:
      
                                
                              #  if lookup_check == 0 and arr[j,c_col] == max(arr[j - (60 * 8):j + 1,c_col]) and arr[j,c_col] - min(arr[j - (60 * 8):j + 1,c_col]) >= .3:
                                #if arr[j,l_col] <= val and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold:
                            
                            arr[j,entry_col] = val - pip_threshold2
                            arr[j,new_col] = 1
                            arr[j,lookup_col] = lookup_ind
                            lookup_check = 1
                            break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator6'] == 1].shape)
    return df

cur = 'NON JPY'
#cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .3
    pip_threshold = .5
    pip_threshold2 = .1
    drop_threshold = .1
else:
    pip_threshold = .003
    pip_threshold2 = .0015

    
    
#df = get_support6(df,lookup_range = 1500,lookup_range2 = 1500,stop_range = 10000,pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2)
#df.head()




In [ ]:
final_list = []
ending_val = 100000

pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]
lookup_ranges = [500,1000,1500,2000,2500]
lookup_range2s = [500,1000,1500,2000,2500]
stop_ranges = [12500,10000,7500,5000,2500]

sls[.0005,.00075,.001,.00125,.0015,.002,.0025]
tps[.0015,.002,.0025,.003,.004,.005,.007,.009]
num_trades = 200
risk_size = .01
for pip_threshold in pip_thresholds:
    for pip_threshold2 in pip_threshold2s:
        for lookup_range in lookup_ranges:
            for lookup_range2 in lookup_range2s:
                for stop_range in stop_ranges:
                    for sl in sls:
                        for tp in tps:
                            


ending_val = strategy_tester_buy6(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 150,risk_size = .02,entry = 'entry_point6',pr = False)
final_list.append([ending_val,pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,sl,tp,num_trades,risk_size])



In [ ]:
c = 0
final_list = []
pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]
lookup_ranges = [500,1000,1500,2000,2500]
lookup_range2s = [500,1000,1500,2000,2500]
stop_ranges = [12500,10000,7500,5000,2500]


pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]
lookup_ranges = [1500,2000,2500]
lookup_range2s = [1000,1500,2000]
stop_ranges = [10000]

sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
tps = [.0015,.002,.0025,.003,.004,.005,.007,.009]
num_trades = 200
risk_size = .01
for pip_threshold in pip_thresholds:
    for pip_threshold2 in pip_threshold2s:
        for lookup_range in lookup_ranges:
            for lookup_range2 in lookup_range2s:
                for stop_range in stop_ranges:
                    c+=1
                    print()
                    print('LOOP NUM:',c)
                    df = get_support6(df,lookup_range = lookup_range,lookup_range2 = lookup_range2,stop_range = stop_range,pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2)
                    for sl in sls:
                        for tp in tps:
                            ending_val = strategy_tester_buy6(df,col = 'support_indicator6',sl = sl,tp = tp,num_trades = 250,risk_size = .01,entry = 'entry_point6',pr = False)
                            final_list.append([ending_val,pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,sl,tp,num_trades,risk_size])
                            
                            
final_list                            


In [ ]:
c = 0
final_list = []
pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]
lookup_ranges = [500,1000,1500,2000,2500]
lookup_range2s = [500,1000,1500,2000,2500]
stop_ranges = [12500,10000,7500,5000,2500]


pip_threshold2s = [.0005,.001,.0015,.002,.0025]
pip_thresholds = [.003,.005,.0075]

lookup_ranges = [1500,2000,2500]
lookup_range2s = [1000,1500,2000]
stop_ranges = [10000]

sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
tps = [.0015,.002,.0025,.003,.004,.005,.007,.009]
num_trades = 200
risk_size = .01
for pip_threshold in pip_thresholds:
    for pip_threshold2 in pip_threshold2s:
        for lookup_range in lookup_ranges:
            for lookup_range2 in lookup_range2s:
                for stop_range in stop_ranges:
                    c+=1
                    print()
                    print('LOOP NUM:',c,)
                    df = get_support6(df,lookup_range = lookup_range,lookup_range2 = lookup_range2,stop_range = stop_range,pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2)
                    for sl in sls:
                        for tp in tps:
                            ending_val = strategy_tester_buy6(df,col = 'support_indicator6',sl = sl,tp = tp,num_trades = 250,risk_size = .01,entry = 'c',pr = False)
                            final_list.append([ending_val,pip_threshold,pip_threshold2,lookup_range,lookup_range2,stop_range,sl,tp,num_trades,risk_size])
                            
                            
final_list                            


In [7]:
df.head()

,time,volume,o,h,l,c,delta,delta_max,delta_vector,max_vector,...,max_o_c,min_o_c,support_lookup,support_indicator,ema_210,ema_750,ema_3000,support_lookup6,support_indicator6,entry_point6
0,2019-01-01 22:00:00,1,1.14649,1.14649,1.14649,1.14649,0,0,0,-0,...,1.14649,1.14649,0,0,1.14649,1.14649,1.14649,0,0,0
1,2019-01-01 22:01:00,1,1.14634,1.14634,1.14634,1.14634,0,0,0,-0,...,1.14634,1.14634,0,0,1.14634,1.14634,1.14634,0,0,0
2,2019-01-01 22:02:00,2,1.14634,1.14634,1.14634,1.14634,0,0,0,-0,...,1.14634,1.14634,0,0,1.14634,1.14634,1.14634,0,0,0
3,2019-01-01 22:03:00,1,1.14658,1.14658,1.14658,1.14658,0,0,0,-0,...,1.14658,1.14658,0,0,1.14658,1.14658,1.14658,0,0,0
4,2019-01-01 22:05:00,2,1.14658,1.14658,1.14635,1.14635,0.00023,0.00023,-0.00023,-0.00023,...,1.14658,1.14635,0,0,1.14635,1.14635,1.14635,0,0,0


In [355]:
ending_val = strategy_tester_buy6(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 150,risk_size = .02,entry = 'entry_point6',pr = False)
val

RUNNING BUY SIMULATOR
SHAPE: (114, 47)

MEDIAN SIM: 19725.0 AVG SIM: 23443.233333333334 WIN RATE 0.14035087719298245  RR: 8.0 NUM TRADES: 150  RISK SIZE: 0.02


19725.0

In [349]:
trade_res,trade_list = strategy_tester_buy6(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 150,risk_size = .02,entry = 'entry_point6')


RUNNING BUY SIMULATOR
SHAPE: (114, 47)
0
200000
400000
600000
800000
1000000
WIN RATE 0.14035087719298245  RR: 8.0 NUM TRADES: 150  RISK SIZE: 0.02
STARTING VAL: 10000  ENDING VAL: 27636
STARTING VAL: 10000  ENDING VAL: 38721
STARTING VAL: 10000  ENDING VAL: 10048
STARTING VAL: 10000  ENDING VAL: 38721
STARTING VAL: 10000  ENDING VAL: 89971
STARTING VAL: 10000  ENDING VAL: 19725
STARTING VAL: 10000  ENDING VAL: 8489
STARTING VAL: 10000  ENDING VAL: 6059
STARTING VAL: 10000  ENDING VAL: 8489
STARTING VAL: 10000  ENDING VAL: 23348
STARTING VAL: 10000  ENDING VAL: 19725
STARTING VAL: 10000  ENDING VAL: 64215
STARTING VAL: 10000  ENDING VAL: 19725
STARTING VAL: 10000  ENDING VAL: 32712
STARTING VAL: 10000  ENDING VAL: 5119
STARTING VAL: 10000  ENDING VAL: 27636
STARTING VAL: 10000  ENDING VAL: 14078
STARTING VAL: 10000  ENDING VAL: 19725
STARTING VAL: 10000  ENDING VAL: 38721
STARTING VAL: 10000  ENDING VAL: 45832
STARTING VAL: 10000  ENDING VAL: 16664
STARTING VAL: 10000  ENDING VAL: 1972

In [344]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 150,entry = 'c') 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 150,entry = 'l') 



RUNNING BUY SIMULATOR
SHAPE: (81, 46)
0
200000
400000
600000
800000
1000000
WIN RATE 0.16049382716049382  RR: 8.0 NUM TRADES: 150
STARTING VAL: 10000  ENDING VAL: 14078
STARTING VAL: 10000  ENDING VAL: 23348
STARTING VAL: 10000  ENDING VAL: 149209
STARTING VAL: 10000  ENDING VAL: 32712
STARTING VAL: 10000  ENDING VAL: 76010
STARTING VAL: 10000  ENDING VAL: 45832
STARTING VAL: 10000  ENDING VAL: 76010
STARTING VAL: 10000  ENDING VAL: 45832
STARTING VAL: 10000  ENDING VAL: 11894
STARTING VAL: 10000  ENDING VAL: 23348
STARTING VAL: 10000  ENDING VAL: 64215
STARTING VAL: 10000  ENDING VAL: 54251
STARTING VAL: 10000  ENDING VAL: 126056
STARTING VAL: 10000  ENDING VAL: 27636
STARTING VAL: 10000  ENDING VAL: 38721
STARTING VAL: 10000  ENDING VAL: 11894
STARTING VAL: 10000  ENDING VAL: 11894
STARTING VAL: 10000  ENDING VAL: 23348
STARTING VAL: 10000  ENDING VAL: 64215
STARTING VAL: 10000  ENDING VAL: 14078
STARTING VAL: 10000  ENDING VAL: 45832
STARTING VAL: 10000  ENDING VAL: 27636
STARTING V

In [342]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 150,entry = 'c') 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 150,entry = 'l') 



RUNNING BUY SIMULATOR
SHAPE: (155, 46)
0
200000
400000
600000
800000
1000000
WIN RATE 0.13548387096774195  RR: 8.0 NUM TRADES: 150
STARTING VAL: 10000  ENDING VAL: 7172
STARTING VAL: 10000  ENDING VAL: 32712
STARTING VAL: 10000  ENDING VAL: 3653
STARTING VAL: 10000  ENDING VAL: 8489
STARTING VAL: 10000  ENDING VAL: 8489
STARTING VAL: 10000  ENDING VAL: 38721
STARTING VAL: 10000  ENDING VAL: 11894
STARTING VAL: 10000  ENDING VAL: 11894
STARTING VAL: 10000  ENDING VAL: 10048
STARTING VAL: 10000  ENDING VAL: 76010
STARTING VAL: 10000  ENDING VAL: 7172
STARTING VAL: 10000  ENDING VAL: 11894
STARTING VAL: 10000  ENDING VAL: 10048
STARTING VAL: 10000  ENDING VAL: 11894
STARTING VAL: 10000  ENDING VAL: 32712
STARTING VAL: 10000  ENDING VAL: 14078
STARTING VAL: 10000  ENDING VAL: 11894
STARTING VAL: 10000  ENDING VAL: 8489
STARTING VAL: 10000  ENDING VAL: 10048
STARTING VAL: 10000  ENDING VAL: 3086
STARTING VAL: 10000  ENDING VAL: 38721
STARTING VAL: 10000  ENDING VAL: 38721
STARTING VAL: 1000

In [314]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400) 


RUNNING BUY SIMULATOR
SHAPE: (105, 46)
0
200000
400000
600000
800000
1000000
WIN RATE 0.1619047619047619  RR: 8.0 NUM TRADES: 400
STARTING VAL: 10000  ENDING VAL: 295257
STARTING VAL: 10000  ENDING VAL: 90694
STARTING VAL: 10000  ENDING VAL: 1594115
STARTING VAL: 10000  ENDING VAL: 178035
STARTING VAL: 10000  ENDING VAL: 46201
STARTING VAL: 10000  ENDING VAL: 2643719
STARTING VAL: 10000  ENDING VAL: 1594115
STARTING VAL: 10000  ENDING VAL: 812067
STARTING VAL: 10000  ENDING VAL: 295257
STARTING VAL: 10000  ENDING VAL: 11989
STARTING VAL: 10000  ENDING VAL: 127069
STARTING VAL: 10000  ENDING VAL: 349488
STARTING VAL: 10000  ENDING VAL: 349488
STARTING VAL: 10000  ENDING VAL: 27858
STARTING VAL: 10000  ENDING VAL: 32975
STARTING VAL: 10000  ENDING VAL: 76621
STARTING VAL: 10000  ENDING VAL: 32975
STARTING VAL: 10000  ENDING VAL: 46201
STARTING VAL: 10000  ENDING VAL: 413680
STARTING VAL: 10000  ENDING VAL: 8557
STARTING VAL: 10000  ENDING VAL: 150409
STARTING VAL: 10000  ENDING VAL: 1780

In [319]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400) 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400) 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400) 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400) 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400) 

RUNNING BUY SIMULATOR
SHAPE: (105, 46)
0
200000
400000
600000
800000
1000000
WIN RATE 0.1619047619047619  RR: 8.0 NUM TRADES: 400
STARTING VAL: 10000  ENDING VAL: 349488
STARTING VAL: 10000  ENDING VAL: 90694
STARTING VAL: 10000  ENDING VAL: 76621
STARTING VAL: 10000  ENDING VAL: 127069
STARTING VAL: 10000  ENDING VAL: 107352
STARTING VAL: 10000  ENDING VAL: 76621
STARTING VAL: 10000  ENDING VAL: 812067
STARTING VAL: 10000  ENDING VAL: 249441
STARTING VAL: 10000  ENDING VAL: 249441
STARTING VAL: 10000  ENDING VAL: 64731
STARTING VAL: 10000  ENDING VAL: 107352
STARTING VAL: 10000  ENDING VAL: 961222
STARTING VAL: 10000  ENDING VAL: 90694
STARTING VAL: 10000  ENDING VAL: 90694
STARTING VAL: 10000  ENDING VAL: 489662
STARTING VAL: 10000  ENDING VAL: 349488
STARTING VAL: 10000  ENDING VAL: 32975
STARTING VAL: 10000  ENDING VAL: 349488
STARTING VAL: 10000  ENDING VAL: 64731
STARTING VAL: 10000  ENDING VAL: 76621
STARTING VAL: 10000  ENDING VAL: 413680
STARTING VAL: 10000  ENDING VAL: 90694


In [323]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0007,tp = .004,num_trades = 400,entry = 'l') 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0007,tp = .004,num_trades = 400,entry = 'l')

RUNNING BUY SIMULATOR
SHAPE: (105, 46)
0
200000
400000
600000
800000
1000000
WIN RATE 0.24761904761904763  RR: 5.714285714285714 NUM TRADES: 400
STARTING VAL: 10000  ENDING VAL: 794635
STARTING VAL: 10000  ENDING VAL: 1328157
STARTING VAL: 10000  ENDING VAL: 367743
STARTING VAL: 10000  ENDING VAL: 4796830
STARTING VAL: 10000  ENDING VAL: 1952365
STARTING VAL: 10000  ENDING VAL: 250169
STARTING VAL: 10000  ENDING VAL: 193505
STARTING VAL: 10000  ENDING VAL: 3263194
STARTING VAL: 10000  ENDING VAL: 323425
STARTING VAL: 10000  ENDING VAL: 323425
STARTING VAL: 10000  ENDING VAL: 149676
STARTING VAL: 10000  ENDING VAL: 323425
STARTING VAL: 10000  ENDING VAL: 1717080
STARTING VAL: 10000  ENDING VAL: 1328157
STARTING VAL: 10000  ENDING VAL: 250169
STARTING VAL: 10000  ENDING VAL: 1717080
STARTING VAL: 10000  ENDING VAL: 1510150
STARTING VAL: 10000  ENDING VAL: 475429
STARTING VAL: 10000  ENDING VAL: 250169
STARTING VAL: 10000  ENDING VAL: 367743
STARTING VAL: 10000  ENDING VAL: 4218751
STARTI

In [333]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 150,entry = 'l') 



RUNNING BUY SIMULATOR
SHAPE: (105, 46)
0
200000
400000
600000
800000
1000000
WIN RATE 0.21904761904761905  RR: 8.0 NUM TRADES: 150
STARTING VAL: 10000  ENDING VAL: 126056
STARTING VAL: 10000  ENDING VAL: 247453
STARTING VAL: 10000  ENDING VAL: 247453
STARTING VAL: 10000  ENDING VAL: 292903
STARTING VAL: 10000  ENDING VAL: 410382
STARTING VAL: 10000  ENDING VAL: 126056
STARTING VAL: 10000  ENDING VAL: 126056
STARTING VAL: 10000  ENDING VAL: 485758
STARTING VAL: 10000  ENDING VAL: 485758
STARTING VAL: 10000  ENDING VAL: 247453
STARTING VAL: 10000  ENDING VAL: 27636
STARTING VAL: 10000  ENDING VAL: 346702
STARTING VAL: 10000  ENDING VAL: 292903
STARTING VAL: 10000  ENDING VAL: 209055
STARTING VAL: 10000  ENDING VAL: 54251
STARTING VAL: 10000  ENDING VAL: 64215
STARTING VAL: 10000  ENDING VAL: 126056
STARTING VAL: 10000  ENDING VAL: 247453
STARTING VAL: 10000  ENDING VAL: 54251
STARTING VAL: 10000  ENDING VAL: 89971
STARTING VAL: 10000  ENDING VAL: 126056
STARTING VAL: 10000  ENDING VAL: 6

In [339]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 150,entry = 'l') 

RUNNING BUY SIMULATOR
SHAPE: (105, 46)
0
200000
400000
600000
800000
1000000
WIN RATE 0.21904761904761905  RR: 8.0 NUM TRADES: 150
STARTING VAL: 10000  ENDING VAL: 176615
STARTING VAL: 10000  ENDING VAL: 76010
STARTING VAL: 10000  ENDING VAL: 346702
STARTING VAL: 10000  ENDING VAL: 1336015
STARTING VAL: 10000  ENDING VAL: 64215
STARTING VAL: 10000  ENDING VAL: 76010
STARTING VAL: 10000  ENDING VAL: 176615
STARTING VAL: 10000  ENDING VAL: 89971
STARTING VAL: 10000  ENDING VAL: 126056
STARTING VAL: 10000  ENDING VAL: 64215
STARTING VAL: 10000  ENDING VAL: 64215
STARTING VAL: 10000  ENDING VAL: 209055
STARTING VAL: 10000  ENDING VAL: 38721
STARTING VAL: 10000  ENDING VAL: 126056
STARTING VAL: 10000  ENDING VAL: 176615
STARTING VAL: 10000  ENDING VAL: 209055
STARTING VAL: 10000  ENDING VAL: 149209
STARTING VAL: 10000  ENDING VAL: 574979
STARTING VAL: 10000  ENDING VAL: 64215
STARTING VAL: 10000  ENDING VAL: 106496
STARTING VAL: 10000  ENDING VAL: 149209
STARTING VAL: 10000  ENDING VAL: 760

In [335]:
np.median([1,2,3])

2.0

In [321]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400,entry = 'l') 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400,entry = 'l') 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400,entry = 'l') 
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .0005,tp = .004,num_trades = 400,entry = 'l') 



RUNNING BUY SIMULATOR
SHAPE: (105, 46)
0
200000
400000
600000
800000
1000000
WIN RATE 0.21904761904761905  RR: 8.0 NUM TRADES: 400
STARTING VAL: 10000  ENDING VAL: 961222
STARTING VAL: 10000  ENDING VAL: 3129300
STARTING VAL: 10000  ENDING VAL: 2643719
STARTING VAL: 10000  ENDING VAL: 489662
STARTING VAL: 10000  ENDING VAL: 39257794
STARTING VAL: 10000  ENDING VAL: 28019608
STARTING VAL: 10000  ENDING VAL: 7271211
STARTING VAL: 10000  ENDING VAL: 16895316
STARTING VAL: 10000  ENDING VAL: 19998537
STARTING VAL: 10000  ENDING VAL: 7271211
STARTING VAL: 10000  ENDING VAL: 33166067
STARTING VAL: 10000  ENDING VAL: 4384409
STARTING VAL: 10000  ENDING VAL: 2643719
STARTING VAL: 10000  ENDING VAL: 33166067
STARTING VAL: 10000  ENDING VAL: 12058756
STARTING VAL: 10000  ENDING VAL: 3129300
STARTING VAL: 10000  ENDING VAL: 8606740
STARTING VAL: 10000  ENDING VAL: 28019608
STARTING VAL: 10000  ENDING VAL: 8606740
STARTING VAL: 10000  ENDING VAL: 1137773
STARTING VAL: 10000  ENDING VAL: 4384409
ST

In [313]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .001,tp = .004,num_trades = 400) 


RUNNING BUY SIMULATOR
SHAPE: (105, 46)
0
200000
400000
600000
800000
1000000
WIN RATE 0.2571428571428571  RR: 4.0 NUM TRADES: 400
STARTING VAL: 10000  ENDING VAL: 220376
STARTING VAL: 10000  ENDING VAL: 17621
STARTING VAL: 10000  ENDING VAL: 294955
STARTING VAL: 10000  ENDING VAL: 101294
STARTING VAL: 10000  ENDING VAL: 62315
STARTING VAL: 10000  ENDING VAL: 46559
STARTING VAL: 10000  ENDING VAL: 325053
STARTING VAL: 10000  ENDING VAL: 242863
STARTING VAL: 10000  ENDING VAL: 62315
STARTING VAL: 10000  ENDING VAL: 101294
STARTING VAL: 10000  ENDING VAL: 181455
STARTING VAL: 10000  ENDING VAL: 68674
STARTING VAL: 10000  ENDING VAL: 149408
STARTING VAL: 10000  ENDING VAL: 123020
STARTING VAL: 10000  ENDING VAL: 123020
STARTING VAL: 10000  ENDING VAL: 83404
STARTING VAL: 10000  ENDING VAL: 123020
STARTING VAL: 10000  ENDING VAL: 123020
STARTING VAL: 10000  ENDING VAL: 21400
STARTING VAL: 10000  ENDING VAL: 42248
STARTING VAL: 10000  ENDING VAL: 68674
STARTING VAL: 10000  ENDING VAL: 62315


In [260]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .001,tp = .002,num_trades = 400) 



RUNNING BUY SIMULATOR
SHAPE: (73, 44)
0
200000
400000
600000
800000
1000000
WIN RATE 0.4383561643835616  RR: 2.0 NUM TRADES: 400
STARTING VAL: 10000  ENDING VAL: 121373
STARTING VAL: 10000  ENDING VAL: 63131
STARTING VAL: 10000  ENDING VAL: 114371
STARTING VAL: 10000  ENDING VAL: 80070
STARTING VAL: 10000  ENDING VAL: 153940
STARTING VAL: 10000  ENDING VAL: 207199
STARTING VAL: 10000  ENDING VAL: 136690
STARTING VAL: 10000  ENDING VAL: 136690
STARTING VAL: 10000  ENDING VAL: 95696
STARTING VAL: 10000  ENDING VAL: 46904
STARTING VAL: 10000  ENDING VAL: 128804
STARTING VAL: 10000  ENDING VAL: 153940
STARTING VAL: 10000  ENDING VAL: 136690
STARTING VAL: 10000  ENDING VAL: 80070
STARTING VAL: 10000  ENDING VAL: 90175
STARTING VAL: 10000  ENDING VAL: 153940
STARTING VAL: 10000  ENDING VAL: 136690
STARTING VAL: 10000  ENDING VAL: 75451
STARTING VAL: 10000  ENDING VAL: 314079
STARTING VAL: 10000  ENDING VAL: 90175
STARTING VAL: 10000  ENDING VAL: 95696
STARTING VAL: 10000  ENDING VAL: 107773


In [267]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .001,tp = .002,num_trades = 400) 



RUNNING BUY SIMULATOR
SHAPE: (73, 44)
0
200000
400000
600000
800000
1000000
WIN RATE 0.4383561643835616  RR: 2.0 NUM TRADES: 400
STARTING VAL: 10000  ENDING VAL: 353715
STARTING VAL: 10000  ENDING VAL: 20413
STARTING VAL: 10000  ENDING VAL: 84973
STARTING VAL: 10000  ENDING VAL: 333309
STARTING VAL: 10000  ENDING VAL: 90175
STARTING VAL: 10000  ENDING VAL: 145059
STARTING VAL: 10000  ENDING VAL: 39245
STARTING VAL: 10000  ENDING VAL: 95696
STARTING VAL: 10000  ENDING VAL: 153940
STARTING VAL: 10000  ENDING VAL: 49775
STARTING VAL: 10000  ENDING VAL: 95696
STARTING VAL: 10000  ENDING VAL: 41648
STARTING VAL: 10000  ENDING VAL: 163365
STARTING VAL: 10000  ENDING VAL: 63131
STARTING VAL: 10000  ENDING VAL: 66996
STARTING VAL: 10000  ENDING VAL: 219885
STARTING VAL: 10000  ENDING VAL: 114371
STARTING VAL: 10000  ENDING VAL: 71098
STARTING VAL: 10000  ENDING VAL: 44198
STARTING VAL: 10000  ENDING VAL: 84973
STARTING VAL: 10000  ENDING VAL: 49775
STARTING VAL: 10000  ENDING VAL: 163365
START

In [269]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .001,tp = .003,num_trades = 400) 

RUNNING BUY SIMULATOR
SHAPE: (73, 44)
0
200000
400000
600000
800000
1000000
WIN RATE 0.3287671232876712  RR: 3.0 NUM TRADES: 400
STARTING VAL: 10000  ENDING VAL: 197552
STARTING VAL: 10000  ENDING VAL: 23742
STARTING VAL: 10000  ENDING VAL: 90133
STARTING VAL: 10000  ENDING VAL: 90133
STARTING VAL: 10000  ENDING VAL: 123368
STARTING VAL: 10000  ENDING VAL: 44480
STARTING VAL: 10000  ENDING VAL: 27777
STARTING VAL: 10000  ENDING VAL: 32497
STARTING VAL: 10000  ENDING VAL: 77041
STARTING VAL: 10000  ENDING VAL: 23742
STARTING VAL: 10000  ENDING VAL: 60881
STARTING VAL: 10000  ENDING VAL: 292469
STARTING VAL: 10000  ENDING VAL: 123368
STARTING VAL: 10000  ENDING VAL: 77041
STARTING VAL: 10000  ENDING VAL: 249988
STARTING VAL: 10000  ENDING VAL: 60881
STARTING VAL: 10000  ENDING VAL: 48111
STARTING VAL: 10000  ENDING VAL: 249988
STARTING VAL: 10000  ENDING VAL: 52038
STARTING VAL: 10000  ENDING VAL: 25681
STARTING VAL: 10000  ENDING VAL: 83330
STARTING VAL: 10000  ENDING VAL: 48111
STARTIN

In [274]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6',sl = .001,tp = .001,num_trades = 400) 

RUNNING BUY SIMULATOR
SHAPE: (73, 44)
0
200000
400000
600000
800000
1000000
WIN RATE 0.6027397260273972  RR: 1.0 NUM TRADES: 400
STARTING VAL: 10000  ENDING VAL: 51563
STARTING VAL: 10000  ENDING VAL: 22258
STARTING VAL: 10000  ENDING VAL: 49541
STARTING VAL: 10000  ENDING VAL: 86736
STARTING VAL: 10000  ENDING VAL: 76927
STARTING VAL: 10000  ENDING VAL: 51563
STARTING VAL: 10000  ENDING VAL: 37441
STARTING VAL: 10000  ENDING VAL: 35972
STARTING VAL: 10000  ENDING VAL: 38969
STARTING VAL: 10000  ENDING VAL: 34562
STARTING VAL: 10000  ENDING VAL: 40559
STARTING VAL: 10000  ENDING VAL: 34562
STARTING VAL: 10000  ENDING VAL: 60511
STARTING VAL: 10000  ENDING VAL: 37441
STARTING VAL: 10000  ENDING VAL: 60511
STARTING VAL: 10000  ENDING VAL: 62980
STARTING VAL: 10000  ENDING VAL: 68227
STARTING VAL: 10000  ENDING VAL: 45731
STARTING VAL: 10000  ENDING VAL: 35972
STARTING VAL: 10000  ENDING VAL: 45731
STARTING VAL: 10000  ENDING VAL: 28296
STARTING VAL: 10000  ENDING VAL: 49541
STARTING VAL:

# TRADE THAT WORKS
## EUR/USD 5 pips below support entry, support is 2-5 days after first reversal
## 10 pip sl and 20 pip tp

# Another trade that works
##     pip_threshold = .003
##    pip_threshold2 = .0015
## EUR/USD support is 2-5 days after first reversal
## 5 pip sl and 40 pip tp

# Idea 6a
## moves up 5 pips from bottom

In [300]:


def get_support6a(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005 ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookup6a'] = 0
    df['support_indicator6a'] = 0
    lookup_col = df.columns.get_loc('support_lookup6a')
    new_col = df.columns.get_loc('support_indicator6a')
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    #for i in range(100000):
    mn_lst = []
    for i in range(arr.shape[0]):

        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
  
                        if arr[j,l_col] <= val and lookup_check == 0: #/- pip_threshold2:
                            for k in range(j,j + 120):
                                local_min = min(arr[j:k + 1,l_col])
                                if arr[k,c_col] >= local_min + pip_threshold2:


                                    arr[k,new_col] = 1
                                    arr[k,lookup_col] = lookup_ind
                                    lookup_check = 1
                                    break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator6a'] == 1].shape)
    return df

cur = 'NON JPY'
#cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .3
    pip_threshold = .5
    pip_threshold2 = .1

else:
    pip_threshold = .003
    pip_threshold2 = .0005

    
    
df = get_support6a(df,lookup_range = 1500,lookup_range2 = 1500,stop_range = 7500,pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2)
df.head()


GETTING SUPPORT INDICATOR
STOP RANGE: 7500
0
200000
400000
600000
800000
1000000
TOTAL FUNCTION TIME: 1.0537320216496786  MINUTES
SHAPE (122, 46)


,time,volume,o,h,l,c,delta,delta_max,delta_vector,max_vector,...,ema_750,ema_3000,support_lookup2,support_indicator2,support_lookup5,support_indicator5,support_lookup6,support_indicator6,support_lookup6a,support_indicator6a
0,2019-01-01 22:00:00,1,1.14649,1.14649,1.14649,1.14649,0,0,0,-0,...,1.14649,1.14649,0,0,0,0,0,0,0,0
1,2019-01-01 22:01:00,1,1.14634,1.14634,1.14634,1.14634,0,0,0,-0,...,1.14634,1.14634,0,0,0,0,0,0,0,0
2,2019-01-01 22:02:00,2,1.14634,1.14634,1.14634,1.14634,0,0,0,-0,...,1.14634,1.14634,0,0,0,0,0,0,0,0
3,2019-01-01 22:03:00,1,1.14658,1.14658,1.14658,1.14658,0,0,0,-0,...,1.14658,1.14658,0,0,0,0,0,0,0,0
4,2019-01-01 22:05:00,2,1.14658,1.14658,1.14635,1.14635,0.00023,0.00023,-0.00023,-0.00023,...,1.14635,1.14635,0,0,0,0,0,0,0,0


In [305]:
trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator6a',sl = .001,tp = .004,num_trades = 500) 


RUNNING BUY SIMULATOR
SHAPE: (122, 46)
0
200000
400000
600000
800000
1000000
WIN RATE 0.1885245901639344  RR: 4.0 NUM TRADES: 500
STARTING VAL: 10000  ENDING VAL: 494
STARTING VAL: 10000  ENDING VAL: 1305
STARTING VAL: 10000  ENDING VAL: 2575
STARTING VAL: 10000  ENDING VAL: 13434
STARTING VAL: 10000  ENDING VAL: 13434
STARTING VAL: 10000  ENDING VAL: 728
STARTING VAL: 10000  ENDING VAL: 3799
STARTING VAL: 10000  ENDING VAL: 6805
STARTING VAL: 10000  ENDING VAL: 5603
STARTING VAL: 10000  ENDING VAL: 3799
STARTING VAL: 10000  ENDING VAL: 19814
STARTING VAL: 10000  ENDING VAL: 12190
STARTING VAL: 10000  ENDING VAL: 975
STARTING VAL: 10000  ENDING VAL: 3128
STARTING VAL: 10000  ENDING VAL: 8264
STARTING VAL: 10000  ENDING VAL: 1584
STARTING VAL: 10000  ENDING VAL: 12190
STARTING VAL: 10000  ENDING VAL: 1584
STARTING VAL: 10000  ENDING VAL: 14804
STARTING VAL: 10000  ENDING VAL: 1074
STARTING VAL: 10000  ENDING VAL: 19814
STARTING VAL: 10000  ENDING VAL: 5603
STARTING VAL: 10000  ENDING VA

In [289]:
def viz_support6a(df,start,end):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240):
        i = ind_list[ind]   
        x = df['support_lookup6a'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point:" + str(df['c'].iloc[i]) + " support at: " + str(df.iloc[x,:]['c']) , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )
        
        fig.add_shape(type='line',
                        x0=df.iloc[i-60,:]['time'],
                        y0=df.iloc[i,:]['c'],
                        x1=df.iloc[i+60,:]['time'],
                        y1=df.iloc[i ,:]['c'],
                        line=dict(color='Blue',),
                        xref='x',
                        yref='y'
        )

        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_210'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_750'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_3000'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator6a'] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 1000)
        i += 1
        
        
viz_support6a(df = df,start = 0,end = 30)

SELL INTO SUPPORT!!!!!
SHAPE: (72, 46)
0
SUPPORT ROW: 6511  SUPPORT NEXT TOUCH: 13345  SUPPORT CLOSE VAL: 1.1423


/opt/anaconda3/lib/python3.8/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




1
SUPPORT ROW: 20272  SUPPORT NEXT TOUCH: 22848  SUPPORT CLOSE VAL: 1.13386
2
SUPPORT ROW: 30778  SUPPORT NEXT TOUCH: 32655  SUPPORT CLOSE VAL: 1.14349
3
SUPPORT ROW: 28540  SUPPORT NEXT TOUCH: 34070  SUPPORT CLOSE VAL: 1.1408200000000002
4
SUPPORT ROW: 40268  SUPPORT NEXT TOUCH: 42430  SUPPORT CLOSE VAL: 1.1258
5
SUPPORT ROW: 42973  SUPPORT NEXT TOUCH: 44606  SUPPORT CLOSE VAL: 1.12501
6
SUPPORT ROW: 74704  SUPPORT NEXT TOUCH: 77426  SUPPORT CLOSE VAL: 1.1336
7
SUPPORT ROW: 77809  SUPPORT NEXT TOUCH: 80814  SUPPORT CLOSE VAL: 1.12744
8
SUPPORT ROW: 120678  SUPPORT NEXT TOUCH: 127840  SUPPORT CLOSE VAL: 1.11745
9
SUPPORT ROW: 145591  SUPPORT NEXT TOUCH: 154427  SUPPORT CLOSE VAL: 1.12202
10
SUPPORT ROW: 191505  SUPPORT NEXT TOUCH: 196869  SUPPORT CLOSE VAL: 1.11014
11
SUPPORT ROW: 202178  SUPPORT NEXT TOUCH: 207270  SUPPORT CLOSE VAL: 1.11701
12
SUPPORT ROW: 207262  SUPPORT NEXT TOUCH: 210282  SUPPORT CLOSE VAL: 1.11626
13
SUPPORT ROW: 219100  SUPPORT NEXT TOUCH: 224805  SUPPORT CLOSE 

In [91]:
"trigger point:" + str(df['c'].iloc[i]) + " support at: " + str(df.iloc[x,:]['c'])

NameError: name 'x' is not defined

In [255]:
def viz_support6(df,start,end):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240):
        i = ind_list[ind]   
        x = df['support_lookup6'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point:" + str(df['c'].iloc[i]) + " support at: " + str(df.iloc[x,:]['c']) , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )
        
        fig.add_shape(type='line',
                        x0=df.iloc[i-60,:]['time'],
                        y0=df.iloc[i,:]['c'],
                        x1=df.iloc[i+60,:]['time'],
                        y1=df.iloc[i ,:]['c'],
                        line=dict(color='Blue',),
                        xref='x',
                        yref='y'
        )

        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_210'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_750'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_3000'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator6'] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 1000)
        i += 1
        
        
viz_support6(df = df,start = 0,end = 30)


SELL INTO SUPPORT!!!!!
SHAPE: (73, 44)
0
SUPPORT ROW: 6511  SUPPORT NEXT TOUCH: 13337  SUPPORT CLOSE VAL: 1.1423


/opt/anaconda3/lib/python3.8/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




1
SUPPORT ROW: 20272  SUPPORT NEXT TOUCH: 22841  SUPPORT CLOSE VAL: 1.13386
2
SUPPORT ROW: 30778  SUPPORT NEXT TOUCH: 32646  SUPPORT CLOSE VAL: 1.14349
3
SUPPORT ROW: 28540  SUPPORT NEXT TOUCH: 34063  SUPPORT CLOSE VAL: 1.1408200000000002
4
SUPPORT ROW: 40268  SUPPORT NEXT TOUCH: 42416  SUPPORT CLOSE VAL: 1.1258
5
SUPPORT ROW: 42973  SUPPORT NEXT TOUCH: 44601  SUPPORT CLOSE VAL: 1.12501
6
SUPPORT ROW: 74704  SUPPORT NEXT TOUCH: 77419  SUPPORT CLOSE VAL: 1.1336
7
SUPPORT ROW: 77809  SUPPORT NEXT TOUCH: 80807  SUPPORT CLOSE VAL: 1.12744
8
SUPPORT ROW: 120678  SUPPORT NEXT TOUCH: 127816  SUPPORT CLOSE VAL: 1.11745
9
SUPPORT ROW: 145591  SUPPORT NEXT TOUCH: 154350  SUPPORT CLOSE VAL: 1.12202
10
SUPPORT ROW: 191505  SUPPORT NEXT TOUCH: 196869  SUPPORT CLOSE VAL: 1.11014
11
SUPPORT ROW: 202178  SUPPORT NEXT TOUCH: 207257  SUPPORT CLOSE VAL: 1.11701
12
SUPPORT ROW: 207262  SUPPORT NEXT TOUCH: 210264  SUPPORT CLOSE VAL: 1.11626
13
SUPPORT ROW: 219100  SUPPORT NEXT TOUCH: 224804  SUPPORT CLOSE 

In [164]:
sum(trade_res)

120

In [165]:
len(trade_res)

138

In [173]:
def strategy_tester(df,col = 'support_indicator6',sl = .1,tp = .5):
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    temp_df = df[df['support_indicator6'] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        if arr[i,col_num] == 1:
            val = arr[i,c_col]
            for j in range(i,i + 10000):
                if i not in trade_list:
                    if arr[j,h_col] > val + sl:
                        trade_res.append(-10)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,l_col] <= val - tp:
                        trade_res.append(50)
                        trade_list.append(i)
                        wins.append(i)
                        break
    print('WIN RATE',len(wins) / len(trade_list))
    return trade_res,trade_list

trade_res,trade_list = strategy_tester(df,col = 'support_indicator6',sl = .1,tp = 1)    


SHAPE: (138, 49)
0
200000
400000
600000
800000
1000000
WIN RATE 0.10869565217391304


# Idea 7
## Goes beyond support then goes the same amount above support
## Sell once it hits the same amount above support

In [186]:
def get_support7(df,lookup_range = 1500,lookup_range2 = 2400,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005 ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    df['support_lookup7'] = 0
    df['support_indicator7'] = 0
    lookup_col = df.columns.get_loc('support_lookup7')
    new_col = df.columns.get_loc('support_indicator7')
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)
    #for i in range(100000):
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range2,c_col])
            
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                max_ = max(arr[i : i + lookup_range2,c_col])
                pip_range = max_ - min_
                if pip_threshold <=  pip_range:
                    
                    for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                        #print('VAL',(max(arr[i : j,c_col]) - min_))
                        if min(arr[i:j-60,l_col]) >= val:
                            if arr[j,l_col] <= val - pip_threshold2 and lookup_check == 0:# and max(arr[j:j + 120,h_col])
                          #  if arr[j,l_col] <= val + pip_threshold2 and lookup_check == 0 and min(arr[i+ 30:j,l_col]) > val:


                                  #  if lookup_check == 0 and arr[j,c_col] == max(arr[j - (60 * 8):j + 1,c_col]) and arr[j,c_col] - min(arr[j - (60 * 8):j + 1,c_col]) >= .3:
                                    #if arr[j,l_col] <= val and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold:

                                arr[j,new_col] = 1
                                arr[j,lookup_col] = lookup_ind
                                lookup_check = 1
                                break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator7'] == 1].shape)
    return df

cur = 'NON JPY'
cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .3
    pip_threshold = .5
    pip_threshold2 = .35
    drop_threshold = .1
else:
    pip_threshold = .005
    drop_threshold = .001
    
df = get_support7(df,lookup_range = 1500,lookup_range2 = 1500,stop_range = 7500,pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2)

GETTING SUPPORT INDICATOR
STOP RANGE: 7500
0
200000
400000
600000
800000
1000000
TOTAL FUNCTION TIME: 4.200794851779937  MINUTES
SHAPE (0, 51)


In [184]:
def viz_support7(df,start,end):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240):
        i = ind_list[ind]   
        x = df['support_lookup7'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point:" + str(df['c'].iloc[i]) + " support at: " + str(df.iloc[x,:]['c']) , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )
        
        fig.add_shape(type='line',
                        x0=df.iloc[i-60,:]['time'],
                        y0=df.iloc[i,:]['c'],
                        x1=df.iloc[i+60,:]['time'],
                        y1=df.iloc[i ,:]['c'],
                        line=dict(color='Blue',),
                        xref='x',
                        yref='y'
        )

        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_210'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_750'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_3000'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator7'] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 1000)
        i += 1
        
        
viz_support7(df = df,start = 60,end = 90)

SELL INTO SUPPORT!!!!!
SHAPE: (125, 51)
60
SUPPORT ROW: 483241  SUPPORT NEXT TOUCH: 487557  SUPPORT CLOSE VAL: 132.406
61
SUPPORT ROW: 491914  SUPPORT NEXT TOUCH: 494103  SUPPORT CLOSE VAL: 132.534
62
SUPPORT ROW: 487660  SUPPORT NEXT TOUCH: 494593  SUPPORT CLOSE VAL: 131.954
63
SUPPORT ROW: 495452  SUPPORT NEXT TOUCH: 502737  SUPPORT CLOSE VAL: 130.69
64
SUPPORT ROW: 502994  SUPPORT NEXT TOUCH: 505227  SUPPORT CLOSE VAL: 130.154
65
SUPPORT ROW: 528754  SUPPORT NEXT TOUCH: 531055  SUPPORT CLOSE VAL: 136.25
66
SUPPORT ROW: 524684  SUPPORT NEXT TOUCH: 531285  SUPPORT CLOSE VAL: 135.986
67
SUPPORT ROW: 532556  SUPPORT NEXT TOUCH: 534278  SUPPORT CLOSE VAL: 133.806
68
SUPPORT ROW: 534284  SUPPORT NEXT TOUCH: 538933  SUPPORT CLOSE VAL: 133.51
69
SUPPORT ROW: 544867  SUPPORT NEXT TOUCH: 547369  SUPPORT CLOSE VAL: 132.688
70
SUPPORT ROW: 561349  SUPPORT NEXT TOUCH: 564587  SUPPORT CLOSE VAL: 134.29399999999998
71
SUPPORT ROW: 572794  SUPPORT NEXT TOUCH: 575957  SUPPORT CLOSE VAL: 135.28799999

In [183]:
def strategy_tester_buy(df,col = 'support_indicator7',sl = .1,tp = .5):
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        if arr[i,col_num] == 1:
            val = arr[i,c_col]
            for j in range(i,i + 10000):
                if i not in trade_list:
                    if arr[j,l_col] < val - sl:
                        trade_res.append(-1 * sl)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,h_col] >= val + tp:
                        trade_res.append(tp)
                        trade_list.append(i)
                        wins.append(i)
                        break
    print('WIN RATE',len(wins) / len(trade_list))
    return trade_res,trade_list

trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator7',sl = .2,tp = .5)    


SHAPE: (125, 51)
0
200000
400000
600000
800000
1000000
WIN RATE 0.328


# Idea 8 double bottoms
## needs to go a certain amount below

In [203]:


def get_support8(df,lookup_range = 1500,stop_range = 2200,pip_threshold = .005,pip_threshold2 = .005  ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookup8'] = 0
    df['support_indicator8'] = 0
    lookup_col = df.columns.get_loc('support_lookup8')
    new_col = df.columns.get_loc('support_indicator8')
    arr = df.values
    min_count = 0
    print('STOP RANGE:',stop_range)

    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            min_ = min(arr[i - lookup_range : i + lookup_range,c_col])
            #max_ = min(arr[i : i + 30,c_col])
            if arr[i,c_col] == min_:
               # print(min_, max(arr[i : i + stop_range,c_col]),'RANGE:',i + 30,i + stop_range)

                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                for j in range(i + lookup_range,i + lookup_range + stop_range):
                    #print('VAL',(max(arr[i : j,c_col]) - min_))
                    if arr[j,l_col] <= val - pip_threshold2 and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold :
                    #if arr[j,l_col] <= val and lookup_check == 0 and (max(arr[i : j,c_col]) - min_) >= pip_threshold:

                        arr[j,new_col] = 1
                        arr[j,lookup_col] = lookup_ind
                        lookup_check = 1
                        break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator8'] == 1].shape)
    return df

cur = 'NON JPY'
cur = 'JPY'


if cur == 'JPY':
    pip_threshold = .3
    pip_threshold = .3
    pip_threshold2 = .1
    drop_threshold = .1
else:
    pip_threshold = .005
    drop_threshold = .001
    
df = get_support8(df,lookup_range = 1000,stop_range = 2500,pip_threshold = pip_threshold,pip_threshold2 = pip_threshold2)
df.head()

GETTING SUPPORT INDICATOR
STOP RANGE: 2500
0
200000
400000
600000
800000
1000000
TOTAL FUNCTION TIME: 0.7818376024564108  MINUTES
SHAPE (154, 53)


,time,volume,o,h,l,c,delta,delta_max,delta_vector,max_vector,...,support_lookup4,support_indicator4,support_lookup5,support_indicator5,support_lookup6,support_indicator6,support_lookup7,support_indicator7,support_lookup8,support_indicator8
0,2019-01-01 22:00:00,1,139.822,139.822,139.822,139.822,0,0,0,-0,...,0,0,0,0,0,0,0,0,0,0
1,2019-01-01 22:01:00,5,139.778,139.824,139.778,139.824,0.046,0.046,0.046,0.046,...,0,0,0,0,0,0,0,0,0,0
2,2019-01-01 22:02:00,3,139.824,139.824,139.806,139.806,0.018,0.018,-0.018,-0.018,...,0,0,0,0,0,0,0,0,0,0
3,2019-01-01 22:03:00,1,139.802,139.802,139.802,139.802,0,0,0,-0,...,0,0,0,0,0,0,0,0,0,0
4,2019-01-01 22:04:00,5,139.806,139.806,139.76,139.76,0.046,0.046,-0.046,-0.046,...,0,0,0,0,0,0,0,0,0,0


In [206]:
def viz_support8(df,start,end):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240):
        i = ind_list[ind]   
        x = df['support_lookup8'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point:" + str(df['c'].iloc[i]) + " support at: " + str(df.iloc[x,:]['c']) , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )
        
        fig.add_shape(type='line',
                        x0=df.iloc[i-60,:]['time'],
                        y0=df.iloc[i,:]['c'],
                        x1=df.iloc[i+60,:]['time'],
                        y1=df.iloc[i ,:]['c'],
                        line=dict(color='Blue',),
                        xref='x',
                        yref='y'
        )

        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_210'],line=dict(color='Yellow',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_750'],line=dict(color='Blue',)))
        fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_3000'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator8'] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 1000)
        i += 1
        
        
viz_support8(df = df,start = 90,end = 120)

SELL INTO SUPPORT!!!!!
SHAPE: (154, 53)
90
SUPPORT ROW: 624521  SUPPORT NEXT TOUCH: 627659  SUPPORT CLOSE VAL: 135.588


/opt/anaconda3/lib/python3.8/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




91
SUPPORT ROW: 626035  SUPPORT NEXT TOUCH: 627668  SUPPORT CLOSE VAL: 135.567
92
SUPPORT ROW: 627668  SUPPORT NEXT TOUCH: 631065  SUPPORT CLOSE VAL: 135.43
93
SUPPORT ROW: 631131  SUPPORT NEXT TOUCH: 633709  SUPPORT CLOSE VAL: 134.61
94
SUPPORT ROW: 633778  SUPPORT NEXT TOUCH: 635102  SUPPORT CLOSE VAL: 133.55
95
SUPPORT ROW: 645146  SUPPORT NEXT TOUCH: 646335  SUPPORT CLOSE VAL: 135.388
96
SUPPORT ROW: 643560  SUPPORT NEXT TOUCH: 646393  SUPPORT CLOSE VAL: 135.192
97
SUPPORT ROW: 655177  SUPPORT NEXT TOUCH: 656746  SUPPORT CLOSE VAL: 137.22799999999998
98
SUPPORT ROW: 653695  SUPPORT NEXT TOUCH: 656920  SUPPORT CLOSE VAL: 136.884
99
SUPPORT ROW: 657786  SUPPORT NEXT TOUCH: 660853  SUPPORT CLOSE VAL: 135.672
100
SUPPORT ROW: 669158  SUPPORT NEXT TOUCH: 670627  SUPPORT CLOSE VAL: 136.314
101
SUPPORT ROW: 672313  SUPPORT NEXT TOUCH: 673793  SUPPORT CLOSE VAL: 134.906
102
SUPPORT ROW: 700709  SUPPORT NEXT TOUCH: 703507  SUPPORT CLOSE VAL: 139.012
103
SUPPORT ROW: 710554  SUPPORT NEXT TOU

In [205]:
def strategy_tester_buy(df,col = 'support_indicator8',sl = .1,tp = .5):
    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)    
    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        if arr[i,col_num] == 1:
            val = arr[i,c_col]
            for j in range(i,i + 10000):
                if i not in trade_list:
                    if arr[j,l_col] < val - sl:
                        trade_res.append(-1 * sl)
                        trade_list.append(i)
                        losses.append(i)
                        break
                    if arr[j,h_col] >= val + tp:
                        trade_res.append(tp)
                        trade_list.append(i)
                        wins.append(i)
                        break
    print('WIN RATE',len(wins) / len(trade_list))
    return trade_res,trade_list

trade_res,trade_list = strategy_tester_buy(df,col = 'support_indicator8',sl = .1,tp = .7)   

SHAPE: (154, 53)
0
200000
400000
600000
800000
1000000
WIN RATE 0.1038961038961039


In [5]:
1500 / 60

25.0

In [7]:
700/60

11.666666666666666

In [189]:
(5 / 2)

2.5

In [195]:
from random import random

def simulate_results(
    num_trades = 200,
    acc_val = 10000,
    rr = 6,
    risk_size = .02,
    win_rate = .2):
    start = acc_val
    for i in range(num_trades):
        rand = random()
        if rand > win_rate:
            acc_val = acc_val - (acc_val * risk_size)
        else:
            acc_val = acc_val + (acc_val * (risk_size * rr))
    print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
    return round(acc_val)
vals = []

num_trades = 500
acc_val = 10000
rr = 4
risk_size = .02
win_rate = .22
for i in range(30):
    
    final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate)
    vals.append(final_val)
print()
print('AVG SIM:',sum(vals) / len(vals))




STARTING VAL: 10000  ENDING VAL: 19814
STARTING VAL: 10000  ENDING VAL: 13434
STARTING VAL: 10000  ENDING VAL: 57697
STARTING VAL: 10000  ENDING VAL: 14804
STARTING VAL: 10000  ENDING VAL: 57697
STARTING VAL: 10000  ENDING VAL: 8264
STARTING VAL: 10000  ENDING VAL: 4613
STARTING VAL: 10000  ENDING VAL: 47507
STARTING VAL: 10000  ENDING VAL: 9107
STARTING VAL: 10000  ENDING VAL: 10037
STARTING VAL: 10000  ENDING VAL: 35495
STARTING VAL: 10000  ENDING VAL: 29226
STARTING VAL: 10000  ENDING VAL: 70073
STARTING VAL: 10000  ENDING VAL: 24064
STARTING VAL: 10000  ENDING VAL: 13434
STARTING VAL: 10000  ENDING VAL: 17980
STARTING VAL: 10000  ENDING VAL: 24064
STARTING VAL: 10000  ENDING VAL: 26520
STARTING VAL: 10000  ENDING VAL: 13434
STARTING VAL: 10000  ENDING VAL: 4186
STARTING VAL: 10000  ENDING VAL: 13434
STARTING VAL: 10000  ENDING VAL: 17980
STARTING VAL: 10000  ENDING VAL: 14804
STARTING VAL: 10000  ENDING VAL: 16315
STARTING VAL: 10000  ENDING VAL: 13434
STARTING VAL: 10000  ENDING V